# EQUITY KE

In [1]:
import pandas as pd

# --- 1. Load the datasets ---
# Load equityHex_May.csv (Internal Records)
equity_hex_df = pd.read_csv('/Users/gracegitau/Downloads/Recon May/equityHex_May.csv')

# Load EquityKe_May.xlsx - Sheet0.csv (Bank Statements)
# We identified that the actual headers are in row index 8 (9th row)
equity_ke_df = pd.read_csv('/Users/gracegitau/Downloads/Recon May/EquityKe_May - Sheet0.csv', header=8)

# --- 2. Preprocessing for equity_hex_df (Internal Records) ---
# Clean column names by stripping whitespace
equity_hex_df.columns = equity_hex_df.columns.str.strip()

# Rename columns for consistency: 'TRANSFER_DATE' to 'Date', 'AMOUNT' to 'Amount'
equity_hex_df = equity_hex_df.rename(columns={'TRANSFER_DATE': 'Date', 'AMOUNT': 'Amount'})

# Convert 'Date' to datetime objects
equity_hex_df['Date'] = pd.to_datetime(equity_hex_df['Date'])

# Select relevant columns for reconciliation. 
equity_hex_df_recon = equity_hex_df[['Date', 'Amount']]

# --- 3. Preprocessing for equity_ke_df (Bank Statements) ---
# Clean column names by stripping whitespace
equity_ke_df.columns = equity_ke_df.columns.str.strip()

# Rename 'Transaction Date' to 'Date' for consistency
equity_ke_df = equity_ke_df.rename(columns={'Transaction Date': 'Date'})

# Convert 'Date' to datetime objects (assuming day-first format like '02-05-2025')
equity_ke_df['Date'] = pd.to_datetime(equity_ke_df['Date'], dayfirst=True)

# Convert 'Credit' to numeric, coercing errors to NaN, then fill NaN with 0
equity_ke_df['Credit'] = pd.to_numeric(equity_ke_df['Credit'], errors='coerce').fillna(0)

# For reconciliation, consider only credit values from the bank statements; ignore debits.
equity_ke_df['Amount'] = equity_ke_df['Credit']

# Select relevant columns for reconciliation.
equity_ke_df_recon = equity_ke_df[['Date', 'Amount']]

# Filter bank records to include only positive amounts (credits)
equity_ke_df_recon = equity_ke_df_recon[equity_ke_df_recon['Amount'] > 0]

# --- NEW FILTER: Filter bank records by 'Narrative' ---
narrative_filter = 'RTGS NALA'
# Ensure the narrative is exactly matched, stripping any extra whitespace from the bank statement 'Narrative' column
equity_tz_bank_df_recon = equity_ke_df_recon[
    equity_ke_df_recon['Narrative'].astype(str).str.strip() == narrative_filter.strip()
].copy()

# --- 4. Calculate Total Amounts and Discrepancy (before reconciliation) ---
total_internal_credits = equity_hex_df_recon['Amount'].sum()
total_bank_credits = equity_ke_df_recon['Amount'].sum()
discrepancy_amount = total_internal_credits - total_bank_credits

print(f"Total Internal Amount:{total_internal_credits:,.2f}")
print(f"Total Bank Credit Amount: {total_bank_credits:,.2f}")
print(f"Overall Discrepancy : {discrepancy_amount:,.2f}\n")


# --- 5. Reconciliation (transaction-level) ---
# Round amounts to 2 decimal places for better matching accuracy
equity_hex_df_recon = equity_hex_df_recon.copy()
equity_ke_df_recon = equity_ke_df_recon.copy()

equity_hex_df_recon['Amount_Rounded'] = equity_hex_df_recon['Amount'].round(2)
equity_ke_df_recon['Amount_Rounded'] = equity_ke_df_recon['Amount'].round(2)

# Perform an outer merge on 'Date' and 'Amount_Rounded' to find matched and unmatched transactions
reconciled_df = pd.merge(
    equity_hex_df_recon.assign(Source_Internal='Internal'), # Add a source column for internal records
    equity_ke_df_recon.assign(Source_Bank='Bank'),         # Add a source column for bank statements
    on=['Date', 'Amount_Rounded'],
    how='outer',
    suffixes=('_Internal', '_Bank')
)

# Identify matched transactions (present in both internal and bank)
matched_transactions = reconciled_df.dropna(subset=['Source_Internal', 'Source_Bank'])

# Identify unmatched internal transactions (present in internal but not in bank)
unmatched_internal = reconciled_df[reconciled_df['Source_Bank'].isna()]

# Identify unmatched bank transactions (present in bank but not in internal)
unmatched_bank = reconciled_df[reconciled_df['Source_Internal'].isna()]

KeyError: 'Narrative'

In [24]:
# --- 6. Summary of Reconciliation ---
print(f"Total Internal Credit Records for Reconciliation: {len(equity_hex_df_recon)}")
print(f"Total Bank Statement Credit Records for Reconciliation: {len(equity_ke_df_recon)}")
print(f"Matched Credit Transactions: {len(matched_transactions)}")
print(f"Unmatched Internal Credit Records: {len(unmatched_internal)}")
print(f"Unmatched Bank Credit Records: {len(unmatched_bank)}")

'''
print("\n--- Matched Credit Transactions (First 5) ---")
print(matched_transactions.head())
'''

Total Internal Credit Records for Reconciliation: 19
Total Bank Statement Credit Records for Reconciliation: 499
Matched Credit Transactions: 18
Unmatched Internal Credit Records: 1
Unmatched Bank Credit Records: 481


'\nprint("\n--- Matched Credit Transactions (First 5) ---")\nprint(matched_transactions.head())\n'

# CELLULANT KE

In [ ]:
import pandas as pd

# --- 1. Load the datasets with correct headers ---
# Load CellulantKeHex_May.csv (Internal Records)
cellulant_hex_df = pd.read_csv('/Users/gracegitau/Downloads/CellulantKeHex_May.csv')

# Load Cellulant_Ke_May - Float Acco.csv (Bank Statements) with header=0
cellulant_ke_df = pd.read_csv('/Users/gracegitau/Downloads/Cellulant_Ke_May - Float Acco.csv', header=5)

# --- 2. Preprocessing for cellulant_hex_df (Internal Records) ---
# Clean column names by stripping whitespace
cellulant_hex_df.columns = cellulant_hex_df.columns.str.strip()

# Rename columns for consistency
cellulant_hex_df = cellulant_hex_df.rename(columns={
    'TRANSFER_DATE': 'Date',
    'AMOUNT': 'Amount',
    'COMMENT': 'Description'
})

# Convert 'Date' to datetime objects
cellulant_hex_df['Date'] = pd.to_datetime(cellulant_hex_df['Date'])

# Filter internal records to include only positive amounts (credits/deposits)
cellulant_hex_df_recon = cellulant_hex_df[cellulant_hex_df['Amount'] > 0].copy()

# Select relevant columns for reconciliation and create a copy to avoid SettingWithCopyWarning
cellulant_hex_df_recon = cellulant_hex_df_recon[['Date', 'Amount', 'Description', 'TRANSFER_ID']].copy()

# !!! NEW FIX: Extract only the date component for matching
cellulant_hex_df_recon.loc[:, 'Date_Match'] = cellulant_hex_df_recon['Date'].dt.date

# --- 3. Preprocessing for cellulant_ke_df (Bank Statements) ---
# Clean column names by stripping whitespace
cellulant_ke_df.columns = cellulant_ke_df.columns.str.strip()

# Rename columns for consistency
cellulant_ke_df = cellulant_ke_df.rename(columns={
    'DateTime': 'Date',
    'Credit Amount': 'Credit',
    'Transaction Type': 'Description',
    'Customer Float Transaction ID': 'ID'
})

# Convert 'Date' to datetime objects (handle format like '5/30/25, 3:21 PM GMT+3')
cellulant_ke_df['Date'] = pd.to_datetime(cellulant_ke_df['Date'], infer_datetime_format=True)

# Remove timezone information to allow merging (if present)
cellulant_ke_df['Date'] = cellulant_ke_df['Date'].dt.tz_localize(None)

# Extract only the date component for matching
cellulant_ke_df.loc[:, 'Date_Match'] = cellulant_ke_df['Date'].dt.date

# Convert 'Credit' to numeric, coercing errors to NaN, then fill NaN with 0
# Remove '+' and ',' before converting to numeric
cellulant_ke_df['Credit'] = cellulant_ke_df['Credit'].astype(str).str.replace('+', '', regex=False).str.replace(',', '', regex=False).astype(float)
cellulant_ke_df['Credit'] = cellulant_ke_df['Credit'].fillna(0)

# For reconciliation, consider only credit values from the bank statements; ignore debits.
cellulant_ke_df['Amount'] = cellulant_ke_df['Credit']

# Filter bank records to include only positive amounts (credits)
cellulant_ke_df_recon = cellulant_ke_df[cellulant_ke_df['Amount'] > 0].copy()

# Select relevant columns for reconciliation and create a copy to avoid SettingWithCopyWarning
cellulant_ke_df_recon = cellulant_ke_df_recon[['Date', 'Amount', 'Description', 'ID', 'Date_Match']].copy()


# --- 4. Calculate Total Amounts and Discrepancy (before reconciliation) ---
total_internal_credits = cellulant_hex_df_recon['Amount'].sum()
total_bank_credits = cellulant_ke_df_recon['Amount'].sum()
discrepancy_amount = total_internal_credits - total_bank_credits

print(f"Total Internal Credit Amount (Cellulant): {total_internal_credits:,.2f}")
print(f"Total Bank Statement Credit Amount (Cellulant): {total_bank_credits:,.2f}")
print(f"Overall Discrepancy (Internal Credits - Bank Credits, Cellulant):{discrepancy_amount:,.2f}\n")

# --- 5. Reconciliation (transaction-level) ---
# Round amounts to 2 decimal places for better matching accuracy
# Use .loc to avoid SettingWithCopyWarning
cellulant_hex_df_recon.loc[:, 'Amount_Rounded'] = cellulant_hex_df_recon['Amount'].round(2)
cellulant_ke_df_recon.loc[:, 'Amount_Rounded'] = cellulant_ke_df_recon['Amount'].round(2)

# Perform an outer merge on 'Date_Match' and 'Amount_Rounded' to find matched and unmatched transactions
reconciled_cellulant_df = pd.merge(
    cellulant_hex_df_recon.assign(Source_Internal='Internal'), # Add a source column for internal records
    cellulant_ke_df_recon.assign(Source_Bank='Bank'),         # Add a source column for bank statements
    on=['Date_Match', 'Amount_Rounded'], # Match on date part only
    how='outer',
    suffixes=('_Internal', '_Bank')
)

# Identify matched transactions
matched_cellulant_transactions = reconciled_cellulant_df.dropna(subset=['Source_Internal', 'Source_Bank'])

# Identify unmatched internal transactions
unmatched_cellulant_internal = reconciled_cellulant_df[reconciled_cellulant_df['Source_Bank'].isna()]

# Identify unmatched bank transactions
unmatched_cellulant_bank = reconciled_cellulant_df[reconciled_cellulant_df['Source_Internal'].isna()]

# --- 6. Summary of Reconciliation ---
print(f"Total Internal Credit Records for Reconciliation: {len(cellulant_hex_df_recon)}")
print(f"Total Bank Statement Credit Records for Reconciliation: {len(cellulant_ke_df_recon)}")
print(f"Matched Credit Transactions: {len(matched_cellulant_transactions)}")
print(f"Unmatched Internal Credit Records: {len(unmatched_cellulant_internal)}")
print(f"Unmatched Bank Credit Records: {len(unmatched_cellulant_bank)}")

print("\n--- Unmatched Internal Credit Records ---")
print(unmatched_cellulant_internal)

Total Internal Credit Amount (Cellulant): 55,368,000.00
Total Bank Statement Credit Amount (Cellulant): 50,368,000.00
Overall Discrepancy (Internal Credits - Bank Credits, Cellulant):5,000,000.00

Total Internal Credit Records for Reconciliation: 10
Total Bank Statement Credit Records for Reconciliation: 9
Matched Credit Transactions: 9
Unmatched Internal Credit Records: 1
Unmatched Bank Credit Records: 0

--- Unmatched Internal Credit Records ---
  Date_Internal  Amount_Internal                       Description_Internal  \
0    2025-05-06        5000000.0  fx-deal-quote-2wfaDKa7EJYRv3NHHD0NMJmuFwp   

                               TRANSFER_ID  Date_Match  Amount_Rounded  \
0  fx-transfer-2y31sVA0iQ5iffnK38eIRZ7TLxo  2025-05-06       5000000.0   

  Source_Internal Date_Bank  Amount_Bank Description_Bank  ID Source_Bank  
0        Internal       NaT          NaN              NaN NaN         NaN  


/var/folders/p2/4fn8fy8n1sj1qf8ygrmybxjc0000n0/T/ipykernel_46529/1634025871.py:47: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  cellulant_ke_df['Date'] = pd.to_datetime(cellulant_ke_df['Date'], infer_datetime_format=True)
/var/folders/p2/4fn8fy8n1sj1qf8ygrmybxjc0000n0/T/ipykernel_46529/1634025871.py:47: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cellulant_ke_df['Date'] = pd.to_datetime(cellulant_ke_df['Date'], infer_datetime_format=True)


# ZAMUPAY (PYCS UBA)

In [22]:
import pandas as pd

# --- 1. Load the datasets with correct headers ---
# Load Zamupay_May - Sheet1.csv (Internal Records)
zamupay_internal_df = pd.read_csv('/Users/gracegitau/Downloads/Recon May/ZamupayHex_May.csv', header=0)

# Load Zamupay_May.xlsx - Sheet1.csv (Bank Statements)
zamupay_bank_df = pd.read_csv('/Users/gracegitau/Downloads/Recon May/Zamupay_May - Sheet1.csv', header=0)

# --- 2. Preprocessing for Zamupay Internal Records ---
# Clean column names by stripping whitespace
zamupay_internal_df.columns = zamupay_internal_df.columns.str.strip()

# Rename columns for consistency
zamupay_internal_df = zamupay_internal_df.rename(columns={
    'TRANSFER_DATE': 'Date',
    'AMOUNT': 'Amount',
})

# Convert 'Date' to datetime objects (assuming MM/DD/YYYY format or inferring)
zamupay_internal_df['Date'] = pd.to_datetime(zamupay_internal_df['Date'])

# Convert 'Amount' to numeric, handling commas and coercing errors
zamupay_internal_df['Amount'] = zamupay_internal_df['Amount'].astype(str).str.replace(',', '', regex=False).astype(float)

# Filter for positive amounts (credits/deposits)
zamupay_internal_df_recon = zamupay_internal_df[zamupay_internal_df['Amount'] > 0].copy()

# Extract only the date component for matching
zamupay_internal_df_recon.loc[:, 'Date_Match'] = zamupay_internal_df_recon['Date'].dt.date


# --- 3. Preprocessing for Zamupay Bank Statements ---
# Clean column names by stripping whitespace
zamupay_bank_df.columns = zamupay_bank_df.columns.str.strip()

# Rename columns for consistency
zamupay_bank_df = zamupay_bank_df.rename(columns={
    'Tran. Date': 'Date',
    'Credit Amt.': 'Amount',
    'Particulars': 'Description'
})

# Convert 'Date' to datetime objects (assuming DD-MM-YYYY or MM/DD/YYYY format)
zamupay_bank_df['Date'] = pd.to_datetime(zamupay_bank_df['Date'])

# Convert 'Amount' to numeric, handling commas and coercing errors
zamupay_bank_df['Amount'] = zamupay_bank_df['Amount'].astype(str).str.replace(',', '', regex=False).astype(float)

# --- Filter out records with "REVERSAL" in 'Description' ---
# Ensure 'Description' column exists and is string type for filtering
if 'Description' in zamupay_bank_df.columns:
    zamupay_bank_df = zamupay_bank_df[
        ~zamupay_bank_df['Description'].astype(str).str.contains('REVERSAL', case=False, na=False)
    ].copy()
else:
    print("Warning: 'Description' (Particulars) column not found in bank statement. Skipping 'REVERSAL' filter.")


# Filter for positive amounts (credits) AFTER reversal filter
zamupay_bank_df_recon = zamupay_bank_df[zamupay_bank_df['Amount'] > 0].copy()

# Extract only the date component for matching
zamupay_bank_df_recon.loc[:, 'Date_Match'] = zamupay_bank_df_recon['Date'].dt.date


# --- 4. Calculate Total Amounts and Discrepancy (before reconciliation) ---
total_internal_credits = zamupay_internal_df_recon['Amount'].sum()
total_bank_credits = zamupay_bank_df_recon['Amount'].sum()
discrepancy_amount = total_internal_credits - total_bank_credits

print(f"Total Internal Credit Amount (Zamupay): ${total_internal_credits:,.2f}")
print(f"Total Bank Statement Credit Amount (Zamupay, after filters): ${total_bank_credits:,.2f}")
print(f"Overall Discrepancy (Internal Credits - Bank Credits, Zamupay): ${discrepancy_amount:,.2f}\n")


# --- 5. Reconciliation (transaction-level, exact date match) ---
# Round amounts to 2 decimal places for better matching accuracy
zamupay_internal_df_recon.loc[:, 'Amount_Rounded'] = zamupay_internal_df_recon['Amount'].round(2)
zamupay_bank_df_recon.loc[:, 'Amount_Rounded'] = zamupay_bank_df_recon['Amount'].round(2)

# Perform an outer merge on 'Date_Match' and 'Amount_Rounded'
reconciled_zamupay_df_exact = pd.merge(
    zamupay_internal_df_recon.assign(Source_Internal='Internal'),
    zamupay_bank_df_recon.assign(Source_Bank='Bank'),
    on=['Date_Match', 'Amount_Rounded'],
    how='outer',
    suffixes=('_Internal', '_Bank')
)

# Identify matched, unmatched internal, and unmatched bank transactions
matched_zamupay_transactions_exact = reconciled_zamupay_df_exact.dropna(subset=['Source_Internal', 'Source_Bank'])
unmatched_zamupay_internal_after_exact = reconciled_zamupay_df_exact[reconciled_zamupay_df_exact['Source_Bank'].isna()].copy()
unmatched_zamupay_bank_after_exact = reconciled_zamupay_df_exact[reconciled_zamupay_df_exact['Source_Internal'].isna()].copy()

# --- 6. Reconciliation with Date Tolerance (3 days) ---
# Define the columns that should be in the matched_with_tolerance DataFrame
matched_tolerance_cols = ['Date_Internal', 'Amount_Internal', 'Date_Match_Internal', 'Source_Internal',
                          'Date_Bank', 'Amount_Bank', 'Date_Match_Bank', 'Source_Bank', 'Amount_Rounded']

matched_with_tolerance_list = [] # Use a list of dictionaries to build the DataFrame

# Convert Date_Match to datetime objects for date arithmetic
unmatched_zamupay_internal_after_exact['Date_Match_DT'] = pd.to_datetime(unmatched_zamupay_internal_after_exact['Date_Match'])
unmatched_zamupay_bank_after_exact['Date_Match_DT'] = pd.to_datetime(unmatched_zamupay_bank_after_exact['Date_Match'])

# Keep track of indices to drop from the original unmatched dataframes
matched_internal_indices = []
matched_bank_indices = []

for i, internal_row in unmatched_zamupay_internal_after_exact.iterrows():
    internal_date = internal_row['Date_Match_DT']
    internal_amount = internal_row['Amount_Rounded']

    # Define date range for tolerance
    start_date = internal_date - pd.Timedelta(days=3)
    end_date = internal_date + pd.Timedelta(days=3)

    # Find potential matches in unmatched bank records within the date tolerance
    potential_matches = unmatched_zamupay_bank_after_exact[
        (unmatched_zamupay_bank_after_exact['Amount_Rounded'] == internal_amount) &
        (unmatched_zamupay_bank_after_exact['Date_Match_DT'] >= start_date) &
        (unmatched_zamupay_bank_after_exact['Date_Match_DT'] <= end_date)
    ]

    if not potential_matches.empty:
        # Take the first match if multiple exist
        matched_bank_row = potential_matches.iloc[0]

        # Construct a new row for matched_with_tolerance
        new_matched_row = {
            'Date_Internal': internal_row['Date_Internal'],
            'Amount_Internal': internal_row['Amount_Internal'],
            'Date_Match_Internal': internal_row['Date_Match'],
            'Source_Internal': 'Internal',
            'Date_Bank': matched_bank_row['Date_Bank'],
            'Amount_Bank': matched_bank_row['Amount_Bank'],
            'Date_Match_Bank': matched_bank_row['Date_Match'],
            'Source_Bank': 'Bank',
            'Amount_Rounded': internal_amount # The amount is the same for both
        }
        matched_with_tolerance_list.append(new_matched_row)

        # Mark indices for removal from unmatched lists
        matched_internal_indices.append(i)
        matched_bank_indices.append(matched_bank_row.name) # Use .name for original index


# Create the DataFrame from the list of dictionaries
matched_zamupay_with_tolerance = pd.DataFrame(matched_with_tolerance_list, columns=matched_tolerance_cols)


# Remove matched records from the unmatched dataframes
current_unmatched_internal = unmatched_zamupay_internal_after_exact.drop(matched_internal_indices).drop(columns=['Date_Match_DT'])
current_unmatched_bank = unmatched_zamupay_bank_after_exact.drop(matched_bank_indices).drop(columns=['Date_Match_DT'])

# --- 7. Reconciliation by Grouping Bank Records (Split Transactions - Refined Logic) ---
matched_by_aggregation_list = []
# Need a temporary copy of current_unmatched_bank to modify while iterating
temp_unmatched_bank_for_agg = current_unmatched_bank.copy()
temp_unmatched_bank_for_agg['Date_Match_DT'] = pd.to_datetime(temp_unmatched_bank_for_agg['Date_Match'])

# Keep track of original bank indices that form part of a matched aggregation
bank_indices_matched_by_agg = []
internal_indices_matched_by_agg = []


for i, internal_row in current_unmatched_internal.iterrows():
    internal_date = pd.to_datetime(internal_row['Date_Match'])
    internal_amount = internal_row['Amount_Rounded']

    # Define date range for tolerance
    start_date = internal_date - pd.Timedelta(days=3)
    end_date = internal_date + pd.Timedelta(days=3)

    # Get potential bank matches within the date tolerance
    potential_bank_records_in_range = temp_unmatched_bank_for_agg[
        (temp_unmatched_bank_for_agg['Date_Match_DT'] >= start_date) &
        (temp_unmatched_bank_for_agg['Date_Match_DT'] <= end_date)
    ]

    # Group these potential bank records by date and sum their amounts
    grouped_bank_sums = potential_bank_records_in_range.groupby('Date_Match_DT')['Amount_Rounded'].sum().reset_index()

    # Find if any aggregated sum matches the internal amount
    matched_agg_bank_entry = grouped_bank_sums[
        grouped_bank_sums['Amount_Rounded'].round(2) == internal_amount
    ]

    if not matched_agg_bank_entry.empty:
        # Take the first aggregated match
        agg_date_dt = matched_agg_bank_entry.iloc[0]['Date_Match_DT']
        agg_amount = matched_agg_bank_entry.iloc[0]['Amount_Rounded']

        # Get the original individual bank records that sum up to this aggregation
        # These are all bank records on the specific agg_date_dt that are still unmatched
        contributing_bank_records = temp_unmatched_bank_for_agg[
            (temp_unmatched_bank_for_agg['Date_Match_DT'] == agg_date_dt)
        ]

        # Double check if the sum of these contributing records still equals the internal amount
        if contributing_bank_records['Amount_Rounded'].sum().round(2) == internal_amount:
            new_matched_row = {
                'Date_Internal': internal_row['Date_Internal'],
                'Amount_Internal': internal_row['Amount_Internal'],
                'Date_Match_Internal': internal_row['Date_Match'],
                'Source_Internal': 'Internal',
                'Date_Bank': None, # This will be set to the aggregation date
                'Amount_Bank': agg_amount,
                'Date_Match_Bank': agg_date_dt.date(),
                'Source_Bank': 'Bank (Aggregated)',
                'Amount_Rounded': internal_amount
            }
            matched_by_aggregation_list.append(new_matched_row)

            # Mark internal index for removal
            internal_indices_matched_by_agg.append(i)

            # Mark all contributing bank records for removal
            bank_indices_matched_by_agg.extend(contributing_bank_records.index.tolist())
            # Remove them from temp_unmatched_bank_for_agg to avoid re-matching
            temp_unmatched_bank_for_agg = temp_unmatched_bank_for_agg.drop(contributing_bank_records.index)


matched_zamupay_by_aggregation = pd.DataFrame(matched_by_aggregation_list, columns=matched_tolerance_cols)

# Remove matched records from the current unmatched dataframes
final_unmatched_zamupay_internal = current_unmatched_internal.drop(internal_indices_matched_by_agg)
# Remove only those bank records that were part of an aggregation
final_unmatched_zamupay_bank = current_unmatched_bank.drop(bank_indices_matched_by_agg, errors='ignore')


# --- 8. Final Summary of Reconciliation ---
total_matched_zamupay = len(matched_zamupay_transactions_exact) + \
                        len(matched_zamupay_with_tolerance) + \
                        len(matched_zamupay_by_aggregation)

total_unmatched_zamupay_internal_final = len(final_unmatched_zamupay_internal)
total_unmatched_zamupay_bank_final = len(final_unmatched_zamupay_bank)

print("\n--- Final Summary of Reconciliation (Zamupay) ---")
print(f"Initial Exact Matched Transactions: {len(matched_zamupay_transactions_exact)}")
print(f"Transactions Matched with 3-day Date Tolerance: {len(matched_zamupay_with_tolerance)}")
print(f"Transactions Matched by Aggregation: {len(matched_zamupay_by_aggregation)}")
print(f"Total Matched Transactions: {total_matched_zamupay}")
print(f"Remaining Unmatched Internal Credit Records: {total_unmatched_zamupay_internal_final}")
print(f"Remaining Unmatched Bank Credit Records: {total_unmatched_zamupay_bank_final}")

if not matched_zamupay_with_tolerance.empty:
    print("\n--- Transactions Matched with 3-day Date Tolerance (Zamupay) ---")
    print(matched_zamupay_with_tolerance[['Date_Match_Internal', 'Amount_Internal', 'Date_Match_Bank', 'Amount_Bank']])

if not matched_zamupay_by_aggregation.empty:
    print("\n--- Transactions Matched by Aggregation (Zamupay) ---")
    print(matched_zamupay_by_aggregation[['Date_Match_Internal', 'Amount_Internal', 'Date_Match_Bank', 'Amount_Bank']])

if not final_unmatched_zamupay_internal.empty:
    print("\n--- Remaining Unmatched Internal Credit Records (Zamupay) ---")
    print(final_unmatched_zamupay_internal[['Date_Match', 'Amount_Internal']])

if not final_unmatched_zamupay_bank.empty:
    print("\n--- Remaining Unmatched Bank Credit Records (Zamupay) ---")
    print(final_unmatched_zamupay_bank[['Date_Match', 'Amount_Bank']])

Total Internal Credit Amount (Zamupay): $866,843,250.00
Total Bank Statement Credit Amount (Zamupay, after filters): $866,843,250.00
Overall Discrepancy (Internal Credits - Bank Credits, Zamupay): $0.00


--- Final Summary of Reconciliation (Zamupay) ---
Initial Exact Matched Transactions: 17
Transactions Matched with 3-day Date Tolerance: 0
Transactions Matched by Aggregation: 1
Total Matched Transactions: 18
Remaining Unmatched Internal Credit Records: 0
Remaining Unmatched Bank Credit Records: 0

--- Transactions Matched by Aggregation (Zamupay) ---
  Date_Match_Internal  Amount_Internal Date_Match_Bank  Amount_Bank
0          2025-05-23      100000000.0      2025-05-23  100000000.0


# SELCOM TZ

In [ ]:
# --- 1. Load the datasets ---
# Load SelcomHex_May.csv (Internal Records)
selcom_hex_df = pd.read_csv('/Users/gracegitau/Downloads/Recon May/SelcomHex_May.csv')

# Load Selcom_May.xlsx - srirmam.csv (Bank Statements)
selcom_bank_df = pd.read_csv('/Users/gracegitau/Downloads/Recon May/Selcom_May - srirmam.csv')

# --- 2. Preprocessing for selcom_hex_df (Internal Records) ---
# Clean column names by stripping whitespace
selcom_hex_df.columns = selcom_hex_df.columns.str.strip()

# Rename columns for consistency: 'TRANSFER_DATE' to 'Date', 'AMOUNT' to 'Amount'
selcom_hex_df = selcom_hex_df.rename(columns={'TRANSFER_DATE': 'Date', 'AMOUNT': 'Amount'})

# Convert 'Date' to datetime objects
selcom_hex_df['Date'] = pd.to_datetime(selcom_hex_df['Date'])

# Filter internal records to include only positive amounts
selcom_hex_df_recon = selcom_hex_df[selcom_hex_df['Amount'] > 0].copy()

# Extract only the date component for matching
selcom_hex_df_recon.loc[:, 'Date_Match'] = selcom_hex_df_recon['Date'].dt.date

# --- 3. Preprocessing for selcom_bank_df (Bank Statements) ---
# Clean column names by stripping whitespace
selcom_bank_df.columns = selcom_bank_df.columns.str.strip()

# Rename columns for consistency: 'DATE' to 'Date', 'AMOUNT' to 'Amount'
selcom_bank_df = selcom_bank_df.rename(columns={'DATE': 'Date', 'AMOUNT': 'Amount'})

# Convert 'Date' to datetime objects (assuming day-first format if not standard)
selcom_bank_df['Date'] = pd.to_datetime(selcom_bank_df['Date'])

# Convert 'Amount' to numeric, coercing errors to NaN, then fill NaN with 0
selcom_bank_df['Amount'] = pd.to_numeric(selcom_bank_df['Amount'], errors='coerce').fillna(0)

# Filter bank records to include only positive amounts (credits)
selcom_bank_df_recon = selcom_bank_df[selcom_bank_df['Amount'] > 0].copy()

# Extract only the date component for matching
selcom_bank_df_recon.loc[:, 'Date_Match'] = selcom_bank_df_recon['Date'].dt.date

# --- 4. Calculate Total Amounts and Discrepancy (before reconciliation) ---
total_internal_credits = selcom_hex_df_recon['Amount'].sum()
total_bank_credits = selcom_bank_df_recon['Amount'].sum()
discrepancy_amount = total_internal_credits - total_bank_credits

print(f"Total Internal Amount: {total_internal_credits:,.2f}")
print(f"Total Bank Statement Credit Amount: {total_bank_credits:,.2f}")
print(f"Overall Discrepancy: {discrepancy_amount:,.2f}\n")

# --- 5. Reconciliation (transaction-level) ---
# Round amounts to 2 decimal places for better matching accuracy
selcom_hex_df_recon.loc[:, 'Amount_Rounded'] = selcom_hex_df_recon['Amount'].round(2)
selcom_bank_df_recon.loc[:, 'Amount_Rounded'] = selcom_bank_df_recon['Amount'].round(2)

# Perform an outer merge on 'Date_Match' and 'Amount_Rounded' to find matched and unmatched transactions
reconciled_selcom_df = pd.merge(
    selcom_hex_df_recon.assign(Source_Internal='Internal'),
    selcom_bank_df_recon.assign(Source_Bank='Bank'),
    on=['Date_Match', 'Amount_Rounded'],
    how='outer',
    suffixes=('_Internal', '_Bank')
)

# Identify matched transactions
matched_selcom_transactions = reconciled_selcom_df.dropna(subset=['Source_Internal', 'Source_Bank'])

# Identify unmatched internal transactions
unmatched_selcom_internal = reconciled_selcom_df[reconciled_selcom_df['Source_Bank'].isna()]

# Identify unmatched bank transactions
unmatched_selcom_bank = reconciled_selcom_df[reconciled_selcom_df['Source_Internal'].isna()]

# --- 6. Summary of Reconciliation ---
print(f"Total Internal Credit Records for Reconciliation (Selcom Tanzania): {len(selcom_hex_df_recon)}")
print(f"Total Bank Statement Credit Records for Reconciliation (Selcom Tanzania): {len(selcom_bank_df_recon)}")
print(f"Matched Credit Transactions (Selcom Tanzania): {len(matched_selcom_transactions)}")
print(f"Unmatched Internal Credit Records (Selcom Tanzania): {len(unmatched_selcom_internal)}")
print(f"Unmatched Bank Credit Records (Selcom Tanzania): {len(unmatched_selcom_bank)}")

print("\n--- Unmatched Internal Credit Records (Selcom Tanzania) ---")
print(unmatched_selcom_internal[['Date_Match', 'Amount_Internal']])

print("\n--- Unmatched Bank Credit Records (Selcom Tanzania) ---")
print(unmatched_selcom_bank[['Date_Match', 'Amount_Bank']])

Total Internal Amount: 711,000,000.00
Total Bank Statement Credit Amount: 0.00
Overall Discrepancy: 711,000,000.00

Total Internal Credit Records for Reconciliation (Selcom Tanzania): 6
Total Bank Statement Credit Records for Reconciliation (Selcom Tanzania): 0
Matched Credit Transactions (Selcom Tanzania): 0
Unmatched Internal Credit Records (Selcom Tanzania): 6
Unmatched Bank Credit Records (Selcom Tanzania): 0

--- Unmatched Internal Credit Records (Selcom Tanzania) ---
   Date_Match  Amount_Internal
0  2025-05-02      150000000.0
1  2025-05-05      161000000.0
2  2025-05-06      100000000.0
3  2025-05-08      100000000.0
4  2025-05-22      100000000.0
5  2025-05-26      100000000.0

--- Unmatched Bank Credit Records (Selcom Tanzania) ---
Empty DataFrame
Columns: [Date_Match, Amount_Bank]
Index: []


# EQUITY TZ

In [34]:
import pandas as pd

# --- 1. Load the datasets ---
# Load EquityTzHex_May.csv (Internal Records)
equity_tz_hex_df = pd.read_csv('/Users/gracegitau/Downloads/Recon May/EquityTzHex_May.csv')

# Load EquityTZ_May - Sheet0.csv (Bank Statements)
# Based on previous examples, the header might be in row index 8 (9th row)
equity_tz_bank_df = pd.read_csv('/Users/gracegitau/Downloads/Recon May/EquityTZ_May - Sheet0.csv', header=8)

# --- 2. Preprocessing for equity_tz_hex_df (Internal Records) ---
# Clean column names by stripping whitespace
equity_tz_hex_df.columns = equity_tz_hex_df.columns.str.strip()

# Rename columns for consistency: 'TRANSFER_DATE' to 'Date', 'AMOUNT' to 'Amount'
equity_tz_hex_df = equity_tz_hex_df.rename(columns={'TRANSFER_DATE': 'Date', 'AMOUNT': 'Amount'})

# Convert 'Date' to datetime objects
equity_tz_hex_df['Date'] = pd.to_datetime(equity_tz_hex_df['Date'])

# Filter internal records to include only positive amounts
equity_tz_hex_df_recon = equity_tz_hex_df[equity_tz_hex_df['Amount'] > 0].copy()

# Extract only the date component for matching
equity_tz_hex_df_recon.loc[:, 'Date_Match'] = equity_tz_hex_df_recon['Date'].dt.date

# --- 3. Preprocessing for equity_tz_bank_df (Bank Statements) ---
# Clean column names by stripping whitespace
equity_tz_bank_df.columns = equity_tz_bank_df.columns.str.strip()

# Rename 'Transaction Date' to 'Date' for consistency
equity_tz_bank_df = equity_tz_bank_df.rename(columns={'Transaction Date': 'Date'})

# Convert 'Date' to datetime objects (assuming day-first format like '02-05-2025')
equity_tz_bank_df['Date'] = pd.to_datetime(equity_tz_bank_df['Date'], dayfirst=True)

# Convert 'Credit' to numeric, coercing errors to NaN, then fill NaN with 0
equity_tz_bank_df['Credit'] = pd.to_numeric(equity_tz_bank_df['Credit'], errors='coerce').fillna(0)

# For reconciliation, consider only credit values from the bank statements; ignore debits.
equity_tz_bank_df['Amount'] = equity_tz_bank_df['Credit']

# Filter bank records to include only positive amounts (credits)
equity_tz_bank_df_recon = equity_tz_bank_df[equity_tz_bank_df['Amount'] > 0].copy()

# --- Filter bank records by 'Narrative' ---
narrative_filter = 'RTGS NALA'
# Ensure the narrative is exactly matched, stripping any extra whitespace from the bank statement 'Narrative' column
equity_tz_bank_df_recon = equity_tz_bank_df_recon[
    equity_tz_bank_df_recon['Narrative'].astype(str).str.strip() == narrative_filter.strip()
].copy()

# Extract only the date component for matching
equity_tz_bank_df_recon.loc[:, 'Date_Match'] = equity_tz_bank_df_recon['Date'].dt.date

# --- 4. Calculate Total Amounts and Discrepancy (before reconciliation) ---
total_internal_credits = equity_tz_hex_df_recon['Amount'].sum()
total_bank_credits = equity_tz_bank_df_recon['Amount'].sum()
discrepancy_amount = total_internal_credits - total_bank_credits

print(f"Total Internal Credit Amount (Equity TZ): {total_internal_credits:,.2f}")
print(f"Total Bank Statement Credit Amount (Equity TZ) (Filtered by Narrative): {total_bank_credits:,.2f}")
print(f"Overall Discrepancy (Internal Credits - Bank Credits, Equity TZ): {discrepancy_amount:,.2f}\n")

# --- 5. Reconciliation (transaction-level) ---
# Round amounts to 2 decimal places for better matching accuracy
equity_tz_hex_df_recon.loc[:, 'Amount_Rounded'] = equity_tz_hex_df_recon['Amount'].round(2)
equity_tz_bank_df_recon.loc[:, 'Amount_Rounded'] = equity_tz_bank_df_recon['Amount'].round(2)

# Perform an outer merge on 'Date_Match' and 'Amount_Rounded' to find matched and unmatched transactions
reconciled_equity_tz_df = pd.merge(
    equity_tz_hex_df_recon.assign(Source_Internal='Internal'),
    equity_tz_bank_df_recon.assign(Source_Bank='Bank'),
    on=['Date_Match', 'Amount_Rounded'],
    how='outer',
    suffixes=('_Internal', '_Bank')
)

# Identify matched transactions
matched_equity_tz_transactions = reconciled_equity_tz_df.dropna(subset=['Source_Internal', 'Source_Bank'])

# Identify unmatched internal transactions
unmatched_equity_tz_internal = reconciled_equity_tz_df[reconciled_equity_tz_df['Source_Bank'].isna()]

# Identify unmatched bank transactions
unmatched_equity_tz_bank = reconciled_equity_tz_df[reconciled_equity_tz_df['Source_Internal'].isna()]

# --- 6. Summary of Reconciliation ---
print(f"Total Internal Credit Records for Reconciliation (Equity TZ): {len(equity_tz_hex_df_recon)}")
print(f"Total Bank Statement Credit Records for Reconciliation (Equity TZ) (Filtered by Narrative): {len(equity_tz_bank_df_recon)}")
print(f"Matched Credit Transactions (Equity TZ): {len(matched_equity_tz_transactions)}")
print(f"Unmatched Internal Credit Records (Equity TZ): {len(unmatched_equity_tz_internal)}")
print(f"Unmatched Bank Credit Records (Equity TZ): {len(unmatched_equity_tz_bank)}")

print("\n--- Unmatched Internal Credit Records (Equity TZ) ---")
print(unmatched_equity_tz_internal[['Date_Match', 'Amount_Internal']])

print("\n--- Unmatched Bank Credit Records (Equity TZ) ---")
print(unmatched_equity_tz_bank[['Date_Match', 'Amount_Bank']])

Total Internal Credit Amount (Equity TZ): 462,800,000.00
Total Bank Statement Credit Amount (Equity TZ) (Filtered by Narrative): 0.00
Overall Discrepancy (Internal Credits - Bank Credits, Equity TZ): 462,800,000.00

Total Internal Credit Records for Reconciliation (Equity TZ): 5
Total Bank Statement Credit Records for Reconciliation (Equity TZ) (Filtered by Narrative): 0
Matched Credit Transactions (Equity TZ): 0
Unmatched Internal Credit Records (Equity TZ): 5
Unmatched Bank Credit Records (Equity TZ): 0

--- Unmatched Internal Credit Records (Equity TZ) ---
   Date_Match  Amount_Internal
0  2025-05-02      100000000.0
1  2025-05-12      100000000.0
2  2025-05-15      100000000.0
3  2025-05-21       89200000.0
4  2025-05-28       73600000.0

--- Unmatched Bank Credit Records (Equity TZ) ---
Empty DataFrame
Columns: [Date_Match, Amount_Bank]
Index: []


# CELLULANT TZ

In [8]:
import pandas as pd

# --- 1. Load the datasets ---
# Load CellulantTzHex_May.csv (Internal Records)
cellulant_tz_hex_df = pd.read_csv('/Users/gracegitau/Downloads/CellulantTzHex_May.csv')

# Load CellulantTz_May - Float Acco.csv (Bank Statements) with header=5 (6th row)
cellulant_tz_bank_df = pd.read_csv('/Users/gracegitau/Downloads/CellulantTz_May - Float Acco.csv', header=5)

# --- 2. Preprocessing for cellulant_tz_hex_df (Internal Records) ---
# Clean column names by stripping whitespace
cellulant_tz_hex_df.columns = cellulant_tz_hex_df.columns.str.strip()

# Rename columns for consistency
cellulant_tz_hex_df = cellulant_tz_hex_df.rename(columns={
    'TRANSFER_DATE': 'Date',
    'AMOUNT': 'Amount',
    'COMMENT': 'Description',
    'TRANSFER_ID': 'ID'
})

# Convert 'Date' to datetime objects
cellulant_tz_hex_df['Date'] = pd.to_datetime(cellulant_tz_hex_df['Date'])

# Filter internal records to include only positive amounts (credits/deposits)
cellulant_tz_hex_df_recon = cellulant_tz_hex_df[cellulant_tz_hex_df['Amount'] > 0].copy()

# Select relevant columns for reconciliation
cellulant_tz_hex_df_recon = cellulant_tz_hex_df_recon[['Date', 'Amount', 'Description', 'ID']].copy()

# Extract only the date component for matching
cellulant_tz_hex_df_recon.loc[:, 'Date_Match'] = cellulant_tz_hex_df_recon['Date'].dt.date
cellulant_tz_hex_df_recon.loc[:, 'Amount_Rounded'] = cellulant_tz_hex_df_recon['Amount'].round(2)
cellulant_tz_hex_df_recon.loc[:, 'Matched_Internal'] = False


# --- 3. Preprocessing for cellulant_tz_bank_df (Bank Statements) ---
# Clean column names by stripping whitespace
cellulant_tz_bank_df.columns = cellulant_tz_bank_df.columns.str.strip()

# Rename columns for consistency
cellulant_tz_bank_df = cellulant_tz_bank_df.rename(columns={
    'DateTime': 'Date',
    'Credit Amount': 'Credit',
    'Transaction Type': 'Description',
    'Customer Float Transaction ID': 'ID'
})

# Convert 'Date' to datetime objects (handle format like '5/30/25, 3:21 PM GMT+3')
cellulant_tz_bank_df['Date'] = pd.to_datetime(cellulant_tz_bank_df['Date'], infer_datetime_format=True)

# Remove timezone information to allow merging (if present)
cellulant_tz_bank_df['Date'] = cellulant_tz_bank_df['Date'].dt.tz_localize(None)

# Extract only the date component for matching
cellulant_tz_bank_df.loc[:, 'Date_Match'] = cellulant_tz_bank_df['Date'].dt.date

# Convert 'Credit' to numeric, coercing errors to NaN, then fill NaN with 0
# Remove '+' and ',' before converting to numeric
cellulant_tz_bank_df['Credit'] = cellulant_tz_bank_df['Credit'].astype(str).str.replace('+', '', regex=False).str.replace(',', '', regex=False).astype(float)
cellulant_tz_bank_df['Credit'] = cellulant_tz_bank_df['Credit'].fillna(0)

# For reconciliation, consider only credit values from the bank statements; ignore debits.
cellulant_tz_bank_df['Amount'] = cellulant_tz_bank_df['Credit']

# Filter bank records to include only positive amounts (credits)
cellulant_tz_bank_df_recon = cellulant_tz_bank_df[cellulant_tz_bank_df['Amount'] > 0].copy()

# Select relevant columns for reconciliation
cellulant_tz_bank_df_recon = cellulant_tz_bank_df_recon[['Date', 'Amount', 'Description', 'ID', 'Date_Match']].copy()
cellulant_tz_bank_df_recon.loc[:, 'Amount_Rounded'] = cellulant_tz_bank_df_recon['Amount'].round(2)
cellulant_tz_bank_df_recon.loc[:, 'Matched_Bank'] = False


# --- 4. Calculate Total Amounts and Discrepancy (before reconciliation) ---
total_internal_credits = cellulant_tz_hex_df_recon['Amount'].sum()
total_bank_credits = cellulant_tz_bank_df_recon['Amount'].sum()
discrepancy_amount = total_internal_credits - total_bank_credits

print(f"Total Internal Credit Amount (Cellulant TZ): {total_internal_credits:,.2f}")
print(f"Total Bank Statement Credit Amount (Cellulant TZ): {total_bank_credits:,.2f}")
print(f"Overall Discrepancy (Internal Credits - Bank Credits, Cellulant TZ): {discrepancy_amount:,.2f}\n")


# --- 5. Reconciliation (transaction-level) with Date Tolerance ---

# Initialize matched dataframes
matched_transactions_exact = pd.DataFrame()
matched_transactions_tolerance = pd.DataFrame()

# Create copies for iteration
temp_internal = cellulant_tz_hex_df_recon.copy()
temp_bank = cellulant_tz_bank_df_recon.copy()

# Try exact match first (Date_Match and Amount_Rounded)
merged_exact = pd.merge(
    temp_internal,
    temp_bank,
    on=['Date_Match', 'Amount_Rounded'],
    how='inner',
    suffixes=('_Internal', '_Bank')
)

if not merged_exact.empty:
    matched_transactions_exact = pd.concat([matched_transactions_exact, merged_exact])
    # Mark matched records in temporary dataframes
    temp_internal.loc[temp_internal['ID'].isin(merged_exact['ID_Internal']), 'Matched_Internal'] = True
    temp_bank.loc[temp_bank['ID'].isin(merged_exact['ID_Bank']), 'Matched_Bank'] = True

# Get remaining unmatched records
unmatched_internal_after_exact = temp_internal[~temp_internal['Matched_Internal']].copy()
unmatched_bank_after_exact = temp_bank[~temp_bank['Matched_Bank']].copy()

# Try matching with date tolerance for remaining unmatched records
# Define date tolerance (e.g., +/- 3 days)
date_tolerance = pd.Timedelta(days=3)

for index_internal, row_internal in unmatched_internal_after_exact.iterrows():
    found_match = False
    for index_bank, row_bank in unmatched_bank_after_exact.iterrows():
        if (row_internal['Amount_Rounded'] == row_bank['Amount_Rounded']) and \
           (abs(row_internal['Date_Match'] - row_bank['Date_Match']) <= date_tolerance):
            # Found a match within tolerance
            matched_row = pd.concat([row_internal.rename(lambda x: x + '_Internal'),
                                     row_bank.rename(lambda x: x + '_Bank')])
            matched_transactions_tolerance = pd.concat([matched_transactions_tolerance, pd.DataFrame([matched_row])], ignore_index=True)
            unmatched_internal_after_exact.loc[index_internal, 'Matched_Internal'] = True
            unmatched_bank_after_exact.loc[index_bank, 'Matched_Bank'] = True
            found_match = True
            break # Move to the next internal record once a match is found
    # If no exact match was found, and no tolerance match, it remains truly unmatched
    # This logic is handled by filtering out 'Matched_' records in the next step

# Filter out records that were matched (either exact or by tolerance)
final_unmatched_internal = unmatched_internal_after_exact[~unmatched_internal_after_exact['Matched_Internal']].copy()
final_unmatched_bank = unmatched_bank_after_exact[~unmatched_bank_after_exact['Matched_Bank']].copy()


# --- 6. Summary of Reconciliation ---
print(f"Total Internal Credit Records for Reconciliation (Cellulant TZ): {len(cellulant_tz_hex_df_recon)}")
print(f"Total Bank Statement Credit Records for Reconciliation (Cellulant TZ): {len(cellulant_tz_bank_df_recon)}")
print(f"Matched Credit Transactions (Exact Date & Amount): {len(matched_transactions_exact)}")
print(f"Matched Credit Transactions (Amount & Date within +/- {date_tolerance.days} days): {len(matched_transactions_tolerance)}")
print(f"Total Matched Credit Transactions: {len(matched_transactions_exact) + len(matched_transactions_tolerance)}")
print(f"Truly Unmatched Internal Credit Records: {len(final_unmatched_internal)}")
print(f"Truly Unmatched Bank Credit Records: {len(final_unmatched_bank)}")

if not matched_transactions_tolerance.empty:
    print(f"\n--- Matched Credit Transactions (Amount & Date within +/- {date_tolerance.days} days) ---")
    print(matched_transactions_tolerance[['Date_Match_Internal', 'Amount_Internal', 'Date_Match_Bank', 'Amount_Bank']])

if not final_unmatched_internal.empty:
    print("\n--- Truly Unmatched Internal Credit Records (Cellulant TZ) ---")
    print(final_unmatched_internal[['Date_Match', 'Amount']])

if not final_unmatched_bank.empty:
    print("\n--- Truly Unmatched Bank Credit Records (Cellulant TZ) ---")
    print(final_unmatched_bank[['Date_Match', 'Amount']])

Total Internal Credit Amount (Cellulant TZ): 433,225,000.00
Total Bank Statement Credit Amount (Cellulant TZ): 433,225,000.00
Overall Discrepancy (Internal Credits - Bank Credits, Cellulant TZ): 0.00

Total Internal Credit Records for Reconciliation (Cellulant TZ): 5
Total Bank Statement Credit Records for Reconciliation (Cellulant TZ): 5
Matched Credit Transactions (Exact Date & Amount): 0
Matched Credit Transactions (Amount & Date within +/- 3 days): 5
Total Matched Credit Transactions: 5
Truly Unmatched Internal Credit Records: 0
Truly Unmatched Bank Credit Records: 0

--- Matched Credit Transactions (Amount & Date within +/- 3 days) ---
  Date_Match_Internal  Amount_Internal Date_Match_Bank  Amount_Bank
0          2025-05-28      100000000.0      2025-05-30  100000000.0
1          2025-05-26      100000000.0      2025-05-27  100000000.0
2          2025-05-20       32600000.0      2025-05-21   32600000.0
3          2025-05-08      100625000.0      2025-05-09  100625000.0
4          

/var/folders/p2/4fn8fy8n1sj1qf8ygrmybxjc0000n0/T/ipykernel_46529/73649587.py:50: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  cellulant_tz_bank_df['Date'] = pd.to_datetime(cellulant_tz_bank_df['Date'], infer_datetime_format=True)
/var/folders/p2/4fn8fy8n1sj1qf8ygrmybxjc0000n0/T/ipykernel_46529/73649587.py:50: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cellulant_tz_bank_df['Date'] = pd.to_datetime(cellulant_tz_bank_df['Date'], infer_datetime_format=True)


# FLUTTERWAVE UGANDA

In [9]:
import pandas as pd

# --- 1. Load the datasets ---
# Load FW.Ug.Hex_May.csv (Internal Records)
internal_df = pd.read_csv('/Users/gracegitau/Downloads/FW.Ug.Hex_May.csv')

# Load FW.Ug_May.csv (Bank Statements)
bank_df = pd.read_csv('/Users/gracegitau/Downloads/FW.Ug_May.csv')

# --- 2. Preprocessing for internal_df (Internal Records) ---
# Clean column names by stripping whitespace
internal_df.columns = internal_df.columns.str.strip()

# Rename columns for consistency: 'TRANSFER_DATE' to 'Date', 'AMOUNT' to 'Amount'
internal_df = internal_df.rename(columns={'TRANSFER_DATE': 'Date', 'AMOUNT': 'Amount'})

# Convert 'Date' to datetime objects
internal_df['Date'] = pd.to_datetime(internal_df['Date'])

# Filter internal records to include only positive amounts
internal_df_recon = internal_df[internal_df['Amount'] > 0].copy()

# Extract only the date component for matching
internal_df_recon.loc[:, 'Date_Match'] = internal_df_recon['Date'].dt.date

# Select relevant columns for reconciliation
internal_df_recon = internal_df_recon[['Date', 'Amount', 'Date_Match']]

# --- 3. Preprocessing for bank_df (Bank Statements) ---
# Clean column names by stripping whitespace
bank_df.columns = bank_df.columns.str.strip()

# Function to find column based on keywords (case-insensitive)
def find_column(df, keywords):
    for col in df.columns:
        if any(keyword.lower() in col.lower() for keyword in keywords):
            return col
    return None

# Dynamically find 'Date' column
date_col_bank = find_column(bank_df, ['date', 'value date', 'transaction date'])
if date_col_bank:
    bank_df = bank_df.rename(columns={date_col_bank: 'Date'})
    bank_df['Date'] = pd.to_datetime(bank_df['Date'], dayfirst=True)
else:
    raise KeyError("Error: 'Date' column not found in bank statement. Please ensure the bank statement contains a date column (e.g., 'Date', 'Value Date', 'Transaction Date').")

# Dynamically find 'Amount' column
amount_col_bank = find_column(bank_df, ['amount', 'credit'])
if amount_col_bank:
    bank_df = bank_df.rename(columns={amount_col_bank: 'Amount'})
    bank_df['Amount'] = pd.to_numeric(bank_df['Amount'], errors='coerce').fillna(0)
else:
    raise KeyError("Error: 'Amount' (or 'Credit') column not found in bank statement. Please ensure the bank statement contains an amount/credit column.")

# Dynamically find 'type' column
type_col_bank = find_column(bank_df, ['type'])
if type_col_bank:
    bank_df = bank_df.rename(columns={type_col_bank: 'Type'})
else:
    print("Warning: 'Type' column not found in bank statement. Proceeding without 'type' filtering.")

# Dynamically find 'remarks' column
remarks_col_bank = find_column(bank_df, ['remarks', 'narration'])
if remarks_col_bank:
    bank_df = bank_df.rename(columns={remarks_col_bank: 'Remarks'})
else:
    print("Warning: 'Remarks' column not found in bank statement. Proceeding without 'rvsl' filtering.")

# Apply the new filters
# Filter for 'Type' = 'C' (Credits) if 'Type' column exists
if 'Type' in bank_df.columns:
    bank_df = bank_df[bank_df['Type'].astype(str).str.upper() == 'C'].copy()

# Filter out records with 'rvsl' in 'Remarks' if 'Remarks' column exists
if 'Remarks' in bank_df.columns:
    bank_df = bank_df[~bank_df['Remarks'].astype(str).str.contains('rvsl', case=False, na=False)].copy()

# Filter bank records to include only positive amounts (credits) after all other filters
bank_df_recon = bank_df[bank_df['Amount'] > 0].copy()

# Extract only the date component for matching
bank_df_recon.loc[:, 'Date_Match'] = bank_df_recon['Date'].dt.date

# Select relevant columns for reconciliation
bank_df_recon = bank_df_recon[['Date', 'Amount', 'Date_Match']]

# --- 4. Calculate Total Amounts and Discrepancy (before reconciliation) ---
total_internal_credits = internal_df_recon['Amount'].sum()
total_bank_credits = bank_df_recon['Amount'].sum()
discrepancy_amount = total_internal_credits - total_bank_credits

print(f"Total Internal Amount: {total_internal_credits:,.2f}")
print(f"Total Bank Credit Amount: {total_bank_credits:,.2f}")
print(f"Overall Discrepancy: {discrepancy_amount:,.2f}\n")

# --- 5. Initial Reconciliation (transaction-level, exact date match) ---
# Round amounts to 2 decimal places for better matching accuracy
internal_df_recon.loc[:, 'Amount_Rounded'] = internal_df_recon['Amount'].round(2)
bank_df_recon.loc[:, 'Amount_Rounded'] = bank_df_recon['Amount'].round(2)

reconciled_df_exact = pd.merge(
    internal_df_recon.assign(Source_Internal='Internal'),
    bank_df_recon.assign(Source_Bank='Bank'),
    on=['Date_Match', 'Amount_Rounded'],
    how='outer',
    suffixes=('_Internal', '_Bank')
)

matched_exact = reconciled_df_exact.dropna(subset=['Source_Internal', 'Source_Bank'])
unmatched_internal_after_exact = reconciled_df_exact[reconciled_df_exact['Source_Bank'].isna()].copy()
unmatched_bank_after_exact = reconciled_df_exact[reconciled_df_exact['Source_Internal'].isna()].copy()

# --- 6. Reconciliation with Date Tolerance (3 days) ---
# Define the columns that should be in the matched_with_tolerance DataFrame
# These columns are derived from the structure of reconciled_df_exact
matched_tolerance_cols = ['Date_Internal', 'Amount_Internal', 'Date_Match_Internal', 'Source_Internal',
                          'Date_Bank', 'Amount_Bank', 'Date_Match_Bank', 'Source_Bank', 'Amount_Rounded']

matched_with_tolerance_list = [] # Use a list of dictionaries to build the DataFrame

# Convert Date_Match to datetime objects for date arithmetic
unmatched_internal_after_exact['Date_Match_DT'] = pd.to_datetime(unmatched_internal_after_exact['Date_Match'])
unmatched_bank_after_exact['Date_Match_DT'] = pd.to_datetime(unmatched_bank_after_exact['Date_Match'])

# Keep track of indices to drop from the original unmatched dataframes
matched_internal_indices = []
matched_bank_indices = []

for i, internal_row in unmatched_internal_after_exact.iterrows():
    internal_date = internal_row['Date_Match_DT']
    internal_amount = internal_row['Amount_Rounded']

    # Define date range for tolerance
    start_date = internal_date - pd.Timedelta(days=3)
    end_date = internal_date + pd.Timedelta(days=3)

    # Find potential matches in unmatched bank records within the date tolerance
    potential_matches = unmatched_bank_after_exact[
        (unmatched_bank_after_exact['Amount_Rounded'] == internal_amount) &
        (unmatched_bank_after_exact['Date_Match_DT'] >= start_date) &
        (unmatched_bank_after_exact['Date_Match_DT'] <= end_date)
    ]

    if not potential_matches.empty:
        # Take the first match if multiple exist
        matched_bank_row = potential_matches.iloc[0]

        # Construct a new row for matched_with_tolerance
        new_matched_row = {
            'Date_Internal': internal_row['Date_Internal'],
            'Amount_Internal': internal_row['Amount_Internal'],
            'Date_Match_Internal': internal_row['Date_Match'],
            'Source_Internal': 'Internal',
            'Date_Bank': matched_bank_row['Date_Bank'],
            'Amount_Bank': matched_bank_row['Amount_Bank'],
            'Date_Match_Bank': matched_bank_row['Date_Match'],
            'Source_Bank': 'Bank',
            'Amount_Rounded': internal_amount # The amount is the same for both
        }
        matched_with_tolerance_list.append(new_matched_row)

        # Mark indices for removal from unmatched lists
        matched_internal_indices.append(i)
        matched_bank_indices.append(matched_bank_row.name) # Use .name for original index


# Create the DataFrame from the list of dictionaries
matched_with_tolerance = pd.DataFrame(matched_with_tolerance_list, columns=matched_tolerance_cols)


# Remove matched records from the unmatched dataframes
final_unmatched_internal = unmatched_internal_after_exact.drop(matched_internal_indices).drop(columns=['Date_Match_DT'])
final_unmatched_bank = unmatched_bank_after_exact.drop(matched_bank_indices).drop(columns=['Date_Match_DT'])

# --- 7. Final Summary of Reconciliation ---
total_matched = len(matched_exact) + len(matched_with_tolerance)
total_unmatched_internal_final = len(final_unmatched_internal)
total_unmatched_bank_final = len(final_unmatched_bank)

print("\n--- Summary of Reconciliation (with 3-day date tolerance) ---")
print(f"Initial Exact Matched Transactions: {len(matched_exact)}")
print(f"Transactions Matched with 3-day Date Tolerance: {len(matched_with_tolerance)}")
print(f"Total Matched Transactions: {total_matched}")
print(f"Remaining Unmatched Internal Credit Records: {total_unmatched_internal_final}")
print(f"Remaining Unmatched Bank Credit Records: {total_unmatched_bank_final}")

if not matched_with_tolerance.empty:
    print("\n--- Transactions Matched with 3-day Date Tolerance ---")
    # Display relevant columns, showing both dates to highlight the tolerance
    print(matched_with_tolerance[['Date_Match_Internal', 'Amount_Internal', 'Date_Match_Bank', 'Amount_Bank']])

if not final_unmatched_internal.empty:
    print("\n--- Remaining Unmatched Internal Credit Records ---")
    print(final_unmatched_internal[['Date_Match', 'Amount_Internal']])

if not final_unmatched_bank.empty:
    print("\n--- Remaining Unmatched Bank Credit Records ---")
    print(final_unmatched_bank[['Date_Match', 'Amount_Bank']])

Total Internal Amount: 1,663,181,500.00
Total Bank Credit Amount: 1,663,181,500.00
Overall Discrepancy: 0.00


--- Summary of Reconciliation (with 3-day date tolerance) ---
Initial Exact Matched Transactions: 2
Transactions Matched with 3-day Date Tolerance: 5
Total Matched Transactions: 7
Remaining Unmatched Internal Credit Records: 0
Remaining Unmatched Bank Credit Records: 0

--- Transactions Matched with 3-day Date Tolerance ---
  Date_Match_Internal  Amount_Internal Date_Match_Bank  Amount_Bank
0          2025-05-06      201800000.0      2025-05-07  201800000.0
1          2025-05-19      176610000.0      2025-05-20  176610000.0
2          2025-05-21      278214000.0      2025-05-22  278214000.0
3          2025-05-22      169595000.0      2025-05-23  169595000.0
4          2025-05-29      106687500.0      2025-05-30  106687500.0


/var/folders/p2/4fn8fy8n1sj1qf8ygrmybxjc0000n0/T/ipykernel_46529/2921377462.py:44: UserWarning: Parsing dates in %Y-%m-%dT%H:%M:%S%z format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  bank_df['Date'] = pd.to_datetime(bank_df['Date'], dayfirst=True)


# NIGERIA

## Cellulant

In [95]:
import pandas as pd
       
# --- 1. Load the datasets ---
# Load Cellulant Nigeria Internal Records (CSV)
cellulant_ng_hex_df = pd.read_csv('/Users/gracegitau/Downloads/#Recon May/NGN/CNL Hex.csv')

# Load Cellulant Nigeria Bank Statements (Excel)
try:
    cellulant_ng_bank_df = pd.read_excel(
        '/Users/gracegitau/Downloads/#Recon May/NGN/CNL Bank.xlsx', 
        engine='openpyxl', header=5
    )
except ImportError:
    try:
        cellulant_ng_bank_df = pd.read_excel(
            '/Users/gracegitau/Downloads/#Recon May/NGN/CNL Bank.xlsx', 
            engine='xlrd', header=5
        )
    except ImportError:
        print("Error: Please install either openpyxl or xlrd package to read Excel files")
        print("Install with: pip install openpyxl xlrd")
        
# --- 2. Preprocessing for cellulant_ng_hex_df (Internal Records) ---
# Clean column names by stripping whitespace
cellulant_ng_hex_df.columns = cellulant_ng_hex_df.columns.str.strip()

# Rename columns for consistency
cellulant_ng_hex_df = cellulant_ng_hex_df.rename(columns={
    'TRANSFER_DATE': 'Date',
    'AMOUNT': 'Amount',
    'COMMENT': 'Description',
    'TRANSFER_ID': 'ID'
})

# Convert 'Date' to datetime objects
cellulant_ng_hex_df['Date'] = pd.to_datetime(cellulant_ng_hex_df['Date'])

# Filter internal records to include only positive amounts (credits/deposits)
cellulant_ng_hex_df_recon = cellulant_ng_hex_df[cellulant_ng_hex_df['Amount'] > 0].copy()

# Select relevant columns for reconciliation
cellulant_ng_hex_df_recon = cellulant_ng_hex_df_recon[['Date', 'Amount', 'Description', 'ID']].copy()

# Extract only the date component for matching
cellulant_ng_hex_df_recon.loc[:, 'Date_Match'] = cellulant_ng_hex_df_recon['Date'].dt.date

# --- 3. Preprocessing for cellulant_ng_bank_df (Bank Statements) ---
# Clean column names by stripping whitespace
cellulant_ng_bank_df.columns = cellulant_ng_bank_df.columns.str.strip()

# Rename columns for consistency
cellulant_ng_bank_df = cellulant_ng_bank_df.rename(columns={
    'DateTime': 'Date',
    'Credit Amount': 'Credit',
    'Transaction Type': 'Transaction_Type',
    'Customer Float Transaction ID': 'ID'
})

# Convert 'Date' to datetime objects (handle format like '5/30/25, 3:21 PM GMT+3')
cellulant_ng_bank_df['Date'] = pd.to_datetime(cellulant_ng_bank_df['Date'], infer_datetime_format=True)

# Remove timezone information to allow merging (if present)
cellulant_ng_bank_df['Date'] = cellulant_ng_bank_df['Date'].dt.tz_localize(None)

# Extract only the date component for matching
cellulant_ng_bank_df.loc[:, 'Date_Match'] = cellulant_ng_bank_df['Date'].dt.date

# --- Nigeria Specific Filters ---
# Filter for Transaction Type = 'allocate' or 'revoke'
cellulant_ng_bank_df = cellulant_ng_bank_df[
    cellulant_ng_bank_df['Transaction_Type'].isin(['allocate', 'revoke'])
].copy()

# Filter for Transaction ID = 1 (if this is a requirement)
# Note: Assuming 'Transaction ID' is a column in the Nigeria data
if 'Transaction ID' in cellulant_ng_bank_df.columns:
    cellulant_ng_bank_df = cellulant_ng_bank_df[
        cellulant_ng_bank_df['Transaction ID'] == 1
    ].copy()

# Process Credit Amount - remove '+' and ',' then convert to numeric
cellulant_ng_bank_df['Credit'] = cellulant_ng_bank_df['Credit'].astype(str).str.replace('+', '', regex=False).str.replace(',', '', regex=False).astype(float)
cellulant_ng_bank_df['Credit'] = cellulant_ng_bank_df['Credit'].fillna(0)

# Process Debit Amount - remove '+' and ',' then convert to numeric
if 'Debit Amount' in cellulant_ng_bank_df.columns:
    cellulant_ng_bank_df['Debit Amount'] = cellulant_ng_bank_df['Debit Amount'].astype(str).str.replace('+', '', regex=False).str.replace(',', '', regex=False).astype(float)
    cellulant_ng_bank_df['Debit Amount'] = cellulant_ng_bank_df['Debit Amount'].fillna(0)

# For reconciliation, consider only credit values from the bank statements; ignore debits.
cellulant_ng_bank_df['Amount'] = cellulant_ng_bank_df['Credit']

# Filter bank records to include only positive amounts (credits)
cellulant_ng_bank_df_recon = cellulant_ng_bank_df[cellulant_ng_bank_df['Amount'] > 0].copy()

# Select relevant columns for reconciliation
cellulant_ng_bank_df_recon = cellulant_ng_bank_df_recon[['Date', 'Amount', 'Transaction_Type', 'ID', 'Date_Match']].copy()

# --- 4. Calculate Total Amounts and Discrepancy (before reconciliation) ---
total_internal_credits = cellulant_ng_hex_df_recon['Amount'].sum()
total_bank_credits = cellulant_ng_bank_df_recon['Amount'].sum()
discrepancy_amount = total_internal_credits - total_bank_credits

print(f"Total Internal Credit Amount (Cellulant NG): {total_internal_credits:,.2f}")
print(f"Total Bank Statement Credit Amount (Cellulant NG): {total_bank_credits:,.2f}")
print(f"Overall Discrepancy (Internal Credits - Bank Credits, Cellulant NG): {discrepancy_amount:,.2f}\n")

# --- 5. Reconciliation (transaction-level) with Date Tolerance ---
# Round amounts to 2 decimal places for better matching accuracy
cellulant_ng_hex_df_recon.loc[:, 'Amount_Rounded'] = cellulant_ng_hex_df_recon['Amount'].round(2)
cellulant_ng_bank_df_recon.loc[:, 'Amount_Rounded'] = cellulant_ng_bank_df_recon['Amount'].round(2)

# Perform an outer merge on 'Date_Match' and 'Amount_Rounded' to find matched and unmatched transactions
reconciled_cellulant_ng_df = pd.merge(
    cellulant_ng_hex_df_recon.assign(Source_Internal='Internal'),
    cellulant_ng_bank_df_recon.assign(Source_Bank='Bank'),
    on=['Date_Match', 'Amount_Rounded'],
    how='outer',
    suffixes=('_Internal', '_Bank')
)

# Identify matched transactions (present in both internal and bank)
matched_cellulant_ng_transactions = reconciled_cellulant_ng_df.dropna(subset=['Source_Internal', 'Source_Bank'])

# Identify unmatched internal transactions (present in internal but not in bank)
unmatched_cellulant_ng_internal = reconciled_cellulant_ng_df[reconciled_cellulant_ng_df['Source_Bank'].isna()]

# Identify unmatched bank transactions (present in bank but not in internal)
unmatched_cellulant_ng_bank = reconciled_cellulant_ng_df[reconciled_cellulant_ng_df['Source_Internal'].isna()]

# --- Additional matching with date tolerance (3 days) ---
date_tolerance = pd.Timedelta(days=3)
matched_with_tolerance = []

# Convert Date_Match to datetime for comparison
unmatched_cellulant_ng_internal['Date_Match_DT'] = pd.to_datetime(unmatched_cellulant_ng_internal['Date_Match'])
unmatched_cellulant_ng_bank['Date_Match_DT'] = pd.to_datetime(unmatched_cellulant_ng_bank['Date_Match'])

for idx, internal_row in unmatched_cellulant_ng_internal.iterrows():
    internal_date = internal_row['Date_Match_DT']
    internal_amount = internal_row['Amount_Rounded']
    
    # Find potential matches in bank records within date tolerance
    potential_matches = unmatched_cellulant_ng_bank[
        (unmatched_cellulant_ng_bank['Amount_Rounded'] == internal_amount) & 
        (abs(unmatched_cellulant_ng_bank['Date_Match_DT'] - internal_date) <= date_tolerance)
    ]
    
    if not potential_matches.empty:
        # Take the first match
        bank_match = potential_matches.iloc[0]
        matched_with_tolerance.append({
            'Date_Internal': internal_row['Date_Internal'],
            'Amount_Internal': internal_row['Amount_Internal'],
            'Date_Match_Internal': internal_row['Date_Match'],
            'Date_Bank': bank_match['Date_Bank'],
            'Amount_Bank': bank_match['Amount_Bank'],
            'Date_Match_Bank': bank_match['Date_Match'],
            'Amount_Rounded': internal_amount
        })
        
        # Remove matched records from unmatched dataframes
        unmatched_cellulant_ng_internal = unmatched_cellulant_ng_internal.drop(idx)
        unmatched_cellulant_ng_bank = unmatched_cellulant_ng_bank.drop(bank_match.name)

# Convert matched_with_tolerance to DataFrame
matched_with_tolerance_df = pd.DataFrame(matched_with_tolerance)

# --- 6. Summary of Reconciliation ---
print(f"Total Internal Credit Records for Reconciliation (Cellulant NG): {len(cellulant_ng_hex_df_recon)}")
print(f"Total Bank Statement Credit Records for Reconciliation (Cellulant NG): {len(cellulant_ng_bank_df_recon)}")
print(f"Exact Matched Credit Transactions: {len(matched_cellulant_ng_transactions)}")
print(f"Transactions Matched with Date Tolerance: {len(matched_with_tolerance_df)}")
print(f"Total Matched Transactions: {len(matched_cellulant_ng_transactions) + len(matched_with_tolerance_df)}")
print(f"Unmatched Internal Credit Records: {len(unmatched_cellulant_ng_internal)}")
print(f"Unmatched Bank Credit Records: {len(unmatched_cellulant_ng_bank)}\n")

if not matched_with_tolerance_df.empty:
    print("\n--- Transactions Matched with Date Tolerance ---")
    print(matched_with_tolerance_df[['Date_Match_Internal', 'Amount_Internal', 'Date_Match_Bank', 'Amount_Bank']])

if not unmatched_cellulant_ng_internal.empty:
    print("\n--- Unmatched Internal Credit Records ---")
    print(unmatched_cellulant_ng_internal[['Date_Match', 'Amount_Internal']])

if not unmatched_cellulant_ng_bank.empty:
    print("\n--- Unmatched Bank Credit Records ---")
    print(unmatched_cellulant_ng_bank[['Date_Match', 'Amount_Bank']])

Total Internal Credit Amount (Cellulant NG): 2,235,500,000.00
Total Bank Statement Credit Amount (Cellulant NG): 2,917,398,000.00
Overall Discrepancy (Internal Credits - Bank Credits, Cellulant NG): -681,898,000.00

Total Internal Credit Records for Reconciliation (Cellulant NG): 6
Total Bank Statement Credit Records for Reconciliation (Cellulant NG): 7
Exact Matched Credit Transactions: 5
Transactions Matched with Date Tolerance: 0
Total Matched Transactions: 5
Unmatched Internal Credit Records: 1
Unmatched Bank Credit Records: 2


--- Unmatched Internal Credit Records ---
   Date_Match  Amount_Internal
2  2025-06-13      481900000.0

--- Unmatched Bank Credit Records ---
   Date_Match  Amount_Bank
1  2025-06-13  481899000.0
3  2025-06-13  681899000.0


/var/folders/p2/4fn8fy8n1sj1qf8ygrmybxjc0000n0/T/ipykernel_37297/528926428.py:60: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  cellulant_ng_bank_df['Date'] = pd.to_datetime(cellulant_ng_bank_df['Date'], infer_datetime_format=True)
/var/folders/p2/4fn8fy8n1sj1qf8ygrmybxjc0000n0/T/ipykernel_37297/528926428.py:60: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cellulant_ng_bank_df['Date'] = pd.to_datetime(cellulant_ng_bank_df['Date'], infer_datetime_format=True)
/var/folders/p2/4fn8fy8n1sj1qf8ygrmybxjc0000n0/T/ipykernel_37297/528926428.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

## Verto

In [73]:
# --- 1. Load the datasets ---
# Load Verto Nigeria Internal Records (CSV)
verto_ng_hex_df = pd.read_csv('/Users/gracegitau/Downloads/#Recon May/NGN/Verto Hex.csv')
verto_ng_bank_df = pd.read_csv('/Users/gracegitau/Downloads/#Recon May/NGN/Verto Bank.csv', header = 8)

# 2. Preprocessing for internal_df (Internal Records)
verto_ng_hex_df.columns = verto_ng_hex_df.columns.str.strip()

# Rename columns for consistency
verto_ng_hex_df = verto_ng_hex_df.rename(columns={
    'TRANSFER_DATE': 'Date',
    'AMOUNT': 'Amount',
    'COMMENT': 'Description',
    'TRANSFER_ID': 'ID'
})

# Convert 'Date' to datetime objects
verto_ng_hex_df['Date'] = pd.to_datetime(verto_ng_hex_df['Date'])

# Filter internal records to include only positive amounts (credits/deposits)
verto_ng_hex_df_recon = verto_ng_hex_df[verto_ng_hex_df['Amount'] > 0].copy()

# Select relevant columns for reconciliation
verto_ng_hex_df_recon = verto_ng_hex_df_recon[['Date', 'Amount', 'Description', 'ID']].copy()

# Extract only the date component for matching
verto_ng_hex_df_recon.loc[:, 'Date_Match'] = verto_ng_hex_df_recon['Date'].dt.date

# 3. Preprocessing for bank_df (Bank Statements)
# Clean column names by stripping whitespace
verto_ng_bank_df.columns = verto_ng_bank_df.columns.str.strip()

# Rename columns for consistency
verto_ng_bank_df = verto_ng_bank_df.rename(columns={
    'Date': 'Date',
    'Credit': 'Credit',
    'Verto Transaction Id': 'Transaction_ID',
    'Comment': 'Description'
})

# Convert 'Date' to datetime - Nigerian format (day/month/year)
verto_ng_bank_df['Date'] = pd.to_datetime(verto_ng_bank_df['Date'], dayfirst=True)

# Verto Specific Filters
# 1. Filter for June transactions (month = 6, year = 2025)
verto_ng_bank_df = verto_ng_bank_df[
    (verto_ng_bank_df['Date'].dt.month == 6) & 
    (verto_ng_bank_df['Date'].dt.year == 2025)
].copy()

# 2. Process Credit Amount - remove commas and convert to numeric
verto_ng_bank_df['Credit'] = (
    verto_ng_bank_df['Credit'].astype(str)
    .str.replace(',', '', regex=False)
    .replace('', '0')  # Handle empty strings
    .astype(float)
)

# 3. Filter for positive credits only
verto_ng_bank_df = verto_ng_bank_df[
    verto_ng_bank_df['Credit'] > 0
].copy()

# For reconciliation, use the credit values
verto_ng_bank_df['Amount'] = verto_ng_bank_df['Credit']

# Select relevant columns for reconciliation
verto_ng_bank_df_recon = verto_ng_bank_df[['Date', 'Amount', 'Description', 'Transaction_ID']].copy()

# Extract only the date component for matching
verto_ng_bank_df_recon.loc[:, 'Date_Match'] = verto_ng_bank_df_recon['Date'].dt.date

# 4. Calculate Total Amounts and Discrepancy (before reconciliation)
total_internal_credits = verto_ng_hex_df_recon['Amount'].sum()
total_bank_credits = verto_ng_bank_df_recon['Amount'].sum()
discrepancy_amount = total_internal_credits - total_bank_credits

print(f"Total Internal Credit Amount (Verto NG): {total_internal_credits:,.2f}")
print(f"Total Bank Statement Credit Amount (Verto NG): {total_bank_credits:,.2f}")
print(f"Overall Discrepancy (Internal Credits - Bank Credits): {discrepancy_amount:,.2f}")

# 5. Reconciliation (transaction-level) with Date Tolerance
print("\nStep 5: Performing transaction-level reconciliation...")
# Round amounts to 2 decimal places for better matching accuracy
verto_ng_hex_df_recon.loc[:, 'Amount_Rounded'] = verto_ng_hex_df_recon['Amount'].round(2)
verto_ng_bank_df_recon.loc[:, 'Amount_Rounded'] = verto_ng_bank_df_recon['Amount'].round(2)

# Perform an outer merge to find matched and unmatched transactions
reconciled_df = pd.merge(
    verto_ng_hex_df_recon.assign(Source_Internal='Internal'),
    verto_ng_bank_df_recon.assign(Source_Bank='Bank'),
    on=['Date_Match', 'Amount_Rounded'],
    how='outer',
    suffixes=('_Internal', '_Bank')
)

# Identify matched transactions
matched_transactions = reconciled_df.dropna(subset=['Source_Internal', 'Source_Bank'])

# Additional matching with 3-day date tolerance
date_tolerance = pd.Timedelta(days=3)
unmatched_internal = reconciled_df[reconciled_df['Source_Bank'].isna()].copy()
unmatched_bank = reconciled_df[reconciled_df['Source_Internal'].isna()].copy()

# Convert Date_Match to datetime for comparison
unmatched_internal['Date_Match_DT'] = pd.to_datetime(unmatched_internal['Date_Match'])
unmatched_bank['Date_Match_DT'] = pd.to_datetime(unmatched_bank['Date_Match'])

matched_with_tolerance = []
matched_internal_indices = []
matched_bank_indices = []

for idx, internal_row in unmatched_internal.iterrows():
    internal_date = internal_row['Date_Match_DT']
    internal_amount = internal_row['Amount_Rounded']
    
    potential_matches = unmatched_bank[
        (unmatched_bank['Amount_Rounded'] == internal_amount) &
        (abs(unmatched_bank['Date_Match_DT'] - internal_date) <= date_tolerance)
    ]
    
    if not potential_matches.empty:
        bank_match = potential_matches.iloc[0]
        matched_with_tolerance.append({
            'Date_Internal': internal_row['Date_Internal'],
            'Amount_Internal': internal_row['Amount_Internal'],
            'Date_Match_Internal': internal_row['Date_Match'],
            'Date_Bank': bank_match['Date_Bank'],
            'Amount_Bank': bank_match['Amount_Bank'],
            'Date_Match_Bank': bank_match['Date_Match'],
            'Amount_Rounded': internal_amount
        })
        matched_internal_indices.append(idx)
        matched_bank_indices.append(bank_match.name)

# Create DataFrame from tolerance matches
matched_with_tolerance_df = pd.DataFrame(matched_with_tolerance)

# Remove matched records from unmatched dataframes
final_unmatched_internal = unmatched_internal.drop(matched_internal_indices)
final_unmatched_bank = unmatched_bank.drop(matched_bank_indices)

# After the initial reconciliation (Step 5), add this additional matching logic:

# 5a. Merge same-day internal transactions before matching with bank
if len(final_unmatched_internal) > 0:
    # Group internal transactions by date and sum amounts
    internal_aggregated = (
        final_unmatched_internal.groupby('Date_Match')
        .agg({'Amount_Internal': 'sum'})
        .reset_index()
    )
    internal_aggregated['Amount_Rounded'] = internal_aggregated['Amount_Internal'].round(2)
    internal_aggregated['Date_Match_DT'] = pd.to_datetime(internal_aggregated['Date_Match'])
    
    # Try matching aggregated internal transactions with bank
    bank_unmatched = final_unmatched_bank.copy()
    bank_unmatched['Date_Match_DT'] = pd.to_datetime(bank_unmatched['Date_Match'])
    
    matched_aggregated = []
    matched_bank_indices = []
    
    for idx, internal_row in internal_aggregated.iterrows():
        internal_date = internal_row['Date_Match_DT']
        internal_amount = internal_row['Amount_Rounded']
        
        potential_matches = bank_unmatched[
            (bank_unmatched['Amount_Rounded'] == internal_amount) &
            (bank_unmatched['Date_Match_DT'] == internal_date)
        ]
        
        if not potential_matches.empty:
            bank_match = potential_matches.iloc[0]
            matched_aggregated.append({
                'Date_Match': internal_row['Date_Match'],
                'Aggregated_Internal_Amount': internal_row['Amount_Internal'],
                'Bank_Amount': bank_match['Amount_Bank'],
                'Bank_Date': bank_match['Date_Match']
            })
            matched_bank_indices.append(bank_match.name)
    
    # Update unmatched records
    if matched_aggregated:
        matched_aggregated_df = pd.DataFrame(matched_aggregated)
        final_unmatched_bank = final_unmatched_bank.drop(matched_bank_indices)
        
        # Remove the matched internal transactions
        matched_dates = matched_aggregated_df['Date_Match'].unique()
        final_unmatched_internal = final_unmatched_internal[
            ~final_unmatched_internal['Date_Match'].isin(matched_dates)
        ]
        
        print("\nAdditional Matches Found After Aggregating Same-Day Internal Transactions:")
        print(matched_aggregated_df)
        
        # Update matched counts
        total_matched += len(matched_aggregated_df)
        total_unmatched_internal = len(final_unmatched_internal)
        total_unmatched_bank = len(final_unmatched_bank)

# Then proceed with your existing Step 6 summary

# 6. Summary of Reconciliation
total_matched = len(matched_transactions) + len(matched_with_tolerance_df)
total_unmatched_internal = len(final_unmatched_internal)
total_unmatched_bank = len(final_unmatched_bank)

print(f"\nReconciliation Summary (Verto Nigeria):")
print(f"Total Internal Credit Records: {len(verto_ng_hex_df_recon)}")
print(f"Total Bank Credit Records: {len(verto_ng_bank_df_recon)}")
print(f"Exact Matched Transactions: {len(matched_transactions)}")
print(f"Transactions Matched with Date Tolerance: {len(matched_with_tolerance_df)}")
print(f"Total Matched Transactions: {total_matched}")
print(f"Remaining Unmatched Internal Records: {total_unmatched_internal}")
print(f"Remaining Unmatched Bank Records: {total_unmatched_bank}")

if not matched_with_tolerance_df.empty:
    print("\nTransactions Matched with Date Tolerance:")
    print(matched_with_tolerance_df[['Date_Match_Internal', 'Amount_Internal', 
                                   'Date_Match_Bank', 'Amount_Bank']])

if not final_unmatched_internal.empty:
    print("\nUnmatched Internal Records:")
    print(final_unmatched_internal[['Date_Match', 'Amount_Internal']].head())

if not final_unmatched_bank.empty:
    print("\nUnmatched Bank Records:")
    print(final_unmatched_bank[['Date_Match', 'Amount_Bank']].head())

Total Internal Credit Amount (Verto NG): 17,143,290,500.00
Total Bank Statement Credit Amount (Verto NG): 17,143,490,500.00
Overall Discrepancy (Internal Credits - Bank Credits): -200,000.00

Step 5: Performing transaction-level reconciliation...

Additional Matches Found After Aggregating Same-Day Internal Transactions:
   Date_Match  Aggregated_Internal_Amount   Bank_Amount   Bank_Date
0  2025-06-12                7.850000e+08  7.850000e+08  2025-06-12
1  2025-06-13                1.481900e+09  1.481900e+09  2025-06-13
2  2025-06-20                1.488200e+09  1.488200e+09  2025-06-20
3  2025-06-27                1.495200e+09  1.495200e+09  2025-06-27
4  2025-06-30                8.552000e+08  8.552000e+08  2025-06-30

Reconciliation Summary (Verto Nigeria):
Total Internal Credit Records: 30
Total Bank Credit Records: 25
Exact Matched Transactions: 19
Transactions Matched with Date Tolerance: 0
Total Matched Transactions: 19
Remaining Unmatched Internal Records: 1
Remaining Unmatche

## Fincra

In [139]:
# 1. Load the datasets
fincra_ng_hex_df = pd.read_csv('/Users/gracegitau/Downloads/#Recon May/NGN/Fincra Hex.csv')

try:
    fincra_ng_bank_df = pd.read_excel(
        '/Users/gracegitau/Downloads/#Recon May/NGN/Fincra Bank.xlsx', 
        engine='openpyxl',
        header=0
    )
except:
    try:
        fincra_ng_bank_df = pd.read_csv(
            '/Users/gracegitau/Downloads/#Recon May/NGN/Fincra Bank.csv',
            header=0
        )
    except Exception as e:
        print("Error loading bank statement:", str(e))
        raise

# 2. Preprocessing for internal_df (Internal Records)
fincra_ng_hex_df.columns = fincra_ng_hex_df.columns.str.strip()

fincra_ng_hex_df = fincra_ng_hex_df.rename(columns={
    'TRANSFER_DATE': 'Date',
    'AMOUNT': 'Amount',
    'COMMENT': 'Description',
    'TRANSFER_ID': 'ID'
})

fincra_ng_hex_df['Date'] = pd.to_datetime(fincra_ng_hex_df['Date'])
fincra_ng_hex_df_recon = fincra_ng_hex_df[fincra_ng_hex_df['Amount'] > 0].copy()
fincra_ng_hex_df_recon = fincra_ng_hex_df_recon[['Date', 'Amount', 'Description', 'ID']].copy()
fincra_ng_hex_df_recon.loc[:, 'Date_Match'] = fincra_ng_hex_df_recon['Date'].dt.date

# 3. Preprocessing for bank_df (Bank Statements) - FINCRA SPECIFIC
fincra_ng_bank_df.columns = fincra_ng_bank_df.columns.str.strip()

fincra_ng_bank_df = fincra_ng_bank_df.rename(columns={
    'Date Initiated': 'Date',
    'Amount Received': 'Credit',
    'Reference': 'Transaction_ID'
})

fincra_ng_bank_df['Date'] = pd.to_datetime(
    fincra_ng_bank_df['Date'], 
    format='%d/%m/%Y, %I:%M:%S %p GMT%z',
    errors='coerce'
)

fincra_ng_bank_df = fincra_ng_bank_df[
    fincra_ng_bank_df['Status'].str.lower() == 'approved'
].copy()

fincra_ng_bank_df['Credit'] = (
    fincra_ng_bank_df['Credit'].astype(str)
    .str.replace(',', '', regex=False)
    .replace('', '0')
    .astype(float)
)

fincra_ng_bank_df = fincra_ng_bank_df[fincra_ng_bank_df['Credit'] > 0].copy()
fincra_ng_bank_df['Amount'] = fincra_ng_bank_df['Credit']
fincra_ng_bank_df_recon = fincra_ng_bank_df[['Date', 'Amount', 'Transaction_ID']].copy()
fincra_ng_bank_df_recon.loc[:, 'Date_Match'] = fincra_ng_bank_df_recon['Date'].dt.date

# 4. Calculate Total Amounts and Discrepancy
total_internal_credits = fincra_ng_hex_df_recon['Amount'].sum()
total_bank_credits = fincra_ng_bank_df_recon['Amount'].sum()
discrepancy_amount = total_internal_credits - total_bank_credits

print(f"Total Internal Credit Amount (Fincra NG): {total_internal_credits:,.2f}")
print(f"Total Bank Statement Credit Amount (Fincra NG): {total_bank_credits:,.2f}")
print(f"Overall Discrepancy (Internal Credits - Bank Credits): {discrepancy_amount:,.2f}")

# 5. Reconciliation (transaction-level) with Date Tolerance
# Ensure Date_Match columns have consistent types (convert both to string)
fincra_ng_hex_df_recon['Date_Match'] = fincra_ng_hex_df_recon['Date_Match'].astype(str)
fincra_ng_bank_df_recon['Date_Match'] = fincra_ng_bank_df_recon['Date_Match'].astype(str)

# Round amounts to 2 decimal places for better matching accuracy
fincra_ng_hex_df_recon['Amount_Rounded'] = fincra_ng_hex_df_recon['Amount'].round(2)
fincra_ng_bank_df_recon['Amount_Rounded'] = fincra_ng_bank_df_recon['Amount'].round(2)

reconciled_df = pd.merge(
    fincra_ng_hex_df_recon.assign(Source_Internal='Internal'),
    fincra_ng_bank_df_recon.assign(Source_Bank='Bank'),
    on=['Date_Match', 'Amount_Rounded'],
    how='outer',
    suffixes=('_Internal', '_Bank')
)

matched_transactions = reconciled_df.dropna(subset=['Source_Internal', 'Source_Bank'])

date_tolerance = pd.Timedelta(days=3)
unmatched_internal = reconciled_df[reconciled_df['Source_Bank'].isna()].copy()
unmatched_bank = reconciled_df[reconciled_df['Source_Internal'].isna()].copy()

unmatched_internal['Date_Match_DT'] = pd.to_datetime(unmatched_internal['Date_Match'])
unmatched_bank['Date_Match_DT'] = pd.to_datetime(unmatched_bank['Date_Match'])

matched_with_tolerance = []
matched_internal_indices = []
matched_bank_indices = []

for idx, internal_row in unmatched_internal.iterrows():
    internal_date = internal_row['Date_Match_DT']
    internal_amount = internal_row['Amount_Rounded']
    
    potential_matches = unmatched_bank[
        (unmatched_bank['Amount_Rounded'] == internal_amount) &
        (abs(unmatched_bank['Date_Match_DT'] - internal_date) <= date_tolerance)
    ]
    
    if not potential_matches.empty:
        bank_match = potential_matches.iloc[0]
        matched_with_tolerance.append({
            'Date_Internal': internal_row['Date_Internal'],
            'Amount_Internal': internal_row['Amount_Internal'],
            'Date_Match_Internal': internal_row['Date_Match'],
            'Date_Bank': bank_match['Date_Bank'],
            'Amount_Bank': bank_match['Amount_Bank'],
            'Date_Match_Bank': bank_match['Date_Match'],
            'Amount_Rounded': internal_amount
        })
        matched_internal_indices.append(idx)
        matched_bank_indices.append(bank_match.name)

matched_with_tolerance_df = pd.DataFrame(matched_with_tolerance)
final_unmatched_internal = unmatched_internal.drop(matched_internal_indices)
final_unmatched_bank = unmatched_bank.drop(matched_bank_indices)

# 6. Summary of Reconciliation
total_matched = len(matched_transactions) + len(matched_with_tolerance_df)
total_unmatched_internal = len(final_unmatched_internal)
total_unmatched_bank = len(final_unmatched_bank)

print(f"\nReconciliation Summary (Fincra Nigeria):")
print(f"Total Internal Credit Records: {len(fincra_ng_hex_df_recon)}")
print(f"Total Bank Credit Records: {len(fincra_ng_bank_df_recon)}")
print(f"Exact Matched Transactions: {len(matched_transactions)}")
print(f"Transactions Matched with Date Tolerance: {len(matched_with_tolerance_df)}")
print(f"Total Matched Transactions: {total_matched}")
print(f"Remaining Unmatched Internal Records: {total_unmatched_internal}")
print(f"Remaining Unmatched Bank Records: {total_unmatched_bank}")

if not matched_with_tolerance_df.empty:
    print("\nTransactions Matched with Date Tolerance:")
    print(matched_with_tolerance_df[['Date_Match_Internal', 'Amount_Internal', 
                                   'Date_Match_Bank', 'Amount_Bank']])

if not final_unmatched_internal.empty:
    print("\nUnmatched Internal Records:")
    print(final_unmatched_internal[['Date_Match', 'Amount_Internal']].head())

if not final_unmatched_bank.empty:
    print("\nUnmatched Bank Records:")
    print(final_unmatched_bank[['Date_Match', 'Amount_Bank']].head())

Total Internal Credit Amount (Fincra NG): 2,586,500,000.00
Total Bank Statement Credit Amount (Fincra NG): 2,586,525,000.00
Overall Discrepancy (Internal Credits - Bank Credits): -25,000.00

Reconciliation Summary (Fincra Nigeria):
Total Internal Credit Records: 5
Total Bank Credit Records: 6
Exact Matched Transactions: 0
Transactions Matched with Date Tolerance: 0
Total Matched Transactions: 0
Remaining Unmatched Internal Records: 5
Remaining Unmatched Bank Records: 6

Unmatched Internal Records:
   Date_Match  Amount_Internal
0  2025-06-02     3.228000e+08
1  2025-06-06     1.176000e+09
2  2025-06-12     3.438000e+08
3  2025-06-19     3.183000e+08
4  2025-06-20     4.256000e+08

Unmatched Bank Records:
  Date_Match  Amount_Bank
5        NaT      25000.0
6        NaT  318300000.0
7        NaT  322800000.0
8        NaT  343800000.0
9        NaT  425600000.0


## Zenith

In [56]:
# 1. Load the datasets
zenith_ng_hex_df = pd.read_csv('/Users/gracegitau/Downloads/#Recon May/NGN/Zenith Hex.csv')
zenith_ng_bank_df = pd.read_csv('/Users/gracegitau/Downloads/#Recon May/NGN/Zenith Bank.csv', header=3)

# 2. Preprocessing for internal_df (Internal Records)
zenith_ng_hex_df.columns = zenith_ng_hex_df.columns.str.strip()

zenith_ng_hex_df = zenith_ng_hex_df.rename(columns={
    'TRANSFER_DATE': 'Date',
    'AMOUNT': 'Amount',
    'COMMENT': 'Description',
    'TRANSFER_ID': 'ID'
})

zenith_ng_hex_df['Date'] = pd.to_datetime(zenith_ng_hex_df['Date'])
zenith_ng_hex_df_recon = zenith_ng_hex_df[zenith_ng_hex_df['Amount'] > 0].copy()
zenith_ng_hex_df_recon = zenith_ng_hex_df_recon[['Date', 'Amount', 'Description', 'ID']].copy()
zenith_ng_hex_df_recon['Date_Match'] = zenith_ng_hex_df_recon['Date'].dt.date

# 3. Preprocessing for bank_df (Bank Statements) - ZENITH SPECIFIC
zenith_ng_bank_df.columns = zenith_ng_bank_df.columns.str.strip()

# First remove any summary rows that don't contain proper dates
zenith_ng_bank_df = zenith_ng_bank_df[
    zenith_ng_bank_df['Effective Date'].str.match(r'\d{2}/\d{2}/\d{4}', na=False)
].copy()

zenith_ng_bank_df = zenith_ng_bank_df.rename(columns={
    'Effective Date': 'Date',
    'Description/Payee/Memo': 'Description',
    'Credit Amount': 'Credit'
})

# Convert 'Date' to datetime - Nigerian format (day/month/year)
zenith_ng_bank_df['Date'] = pd.to_datetime(zenith_ng_bank_df['Date'], dayfirst=True)

# Zenith Nigeria Specific Filters
# 1. Filter for transactions with Description containing 'TRF'
zenith_ng_bank_df = zenith_ng_bank_df[
    zenith_ng_bank_df['Description'].str.contains('TRF FRM NALA PAYMENTS', case=False, na=False)
].copy()

# 2. Process Credit Amount - remove commas and convert to numeric
zenith_ng_bank_df['Credit'] = (
    zenith_ng_bank_df['Credit'].astype(str)
    .str.replace(',', '', regex=False)
    .replace('', '0')
    .astype(float)
)

# 3. Filter for positive credits only
zenith_ng_bank_df = zenith_ng_bank_df[
    zenith_ng_bank_df['Credit'] > 0
].copy()

zenith_ng_bank_df['Amount'] = zenith_ng_bank_df['Credit']
zenith_ng_bank_df_recon = zenith_ng_bank_df[['Date', 'Amount', 'Description']].copy()
zenith_ng_bank_df_recon['Date_Match'] = zenith_ng_bank_df_recon['Date'].dt.date

# 4. Calculate Total Amounts and Discrepancy
total_internal_credits = zenith_ng_hex_df_recon['Amount'].sum()
total_bank_credits = zenith_ng_bank_df_recon['Amount'].sum()
discrepancy_amount = total_internal_credits - total_bank_credits

print(f"Total Internal Credit Amount (Zenith NG): {total_internal_credits:,.2f}")
print(f"Total Bank Statement Credit Amount (Zenith NG): {total_bank_credits:,.2f}")
print(f"Overall Discrepancy (Internal Credits - Bank Credits): {discrepancy_amount:,.2f}")

# 5. Reconciliation (transaction-level) with Date Tolerance
# Ensure Date_Match columns have consistent types
zenith_ng_hex_df_recon['Date_Match'] = zenith_ng_hex_df_recon['Date_Match'].astype(str)
zenith_ng_bank_df_recon['Date_Match'] = zenith_ng_bank_df_recon['Date_Match'].astype(str)

zenith_ng_hex_df_recon['Amount_Rounded'] = zenith_ng_hex_df_recon['Amount'].round(2)
zenith_ng_bank_df_recon['Amount_Rounded'] = zenith_ng_bank_df_recon['Amount'].round(2)

reconciled_df = pd.merge(
    zenith_ng_hex_df_recon.assign(Source_Internal='Internal'),
    zenith_ng_bank_df_recon.assign(Source_Bank='Bank'),
    on=['Date_Match', 'Amount_Rounded'],
    how='outer',
    suffixes=('_Internal', '_Bank')
)

matched_transactions = reconciled_df.dropna(subset=['Source_Internal', 'Source_Bank'])

date_tolerance = pd.Timedelta(days=3)
unmatched_internal = reconciled_df[reconciled_df['Source_Bank'].isna()].copy()
unmatched_bank = reconciled_df[reconciled_df['Source_Internal'].isna()].copy()

unmatched_internal['Date_Match_DT'] = pd.to_datetime(unmatched_internal['Date_Match'])
unmatched_bank['Date_Match_DT'] = pd.to_datetime(unmatched_bank['Date_Match'])

matched_with_tolerance = []
matched_internal_indices = []
matched_bank_indices = []

for idx, internal_row in unmatched_internal.iterrows():
    internal_date = internal_row['Date_Match_DT']
    internal_amount = internal_row['Amount_Rounded']
    
    potential_matches = unmatched_bank[
        (unmatched_bank['Amount_Rounded'] == internal_amount) &
        (abs(unmatched_bank['Date_Match_DT'] - internal_date) <= date_tolerance)
    ]
    
    if not potential_matches.empty:
        bank_match = potential_matches.iloc[0]
        matched_with_tolerance.append({
            'Date_Internal': internal_row['Date_Internal'],
            'Amount_Internal': internal_row['Amount_Internal'],
            'Date_Match_Internal': internal_row['Date_Match'],
            'Date_Bank': bank_match['Date_Bank'],
            'Amount_Bank': bank_match['Amount_Bank'],
            'Date_Match_Bank': bank_match['Date_Match'],
            'Amount_Rounded': internal_amount
        })
        matched_internal_indices.append(idx)
        matched_bank_indices.append(bank_match.name)

matched_with_tolerance_df = pd.DataFrame(matched_with_tolerance)
final_unmatched_internal = unmatched_internal.drop(matched_internal_indices)
final_unmatched_bank = unmatched_bank.drop(matched_bank_indices)

# 6. Summary of Reconciliation
total_matched = len(matched_transactions) + len(matched_with_tolerance_df)
total_unmatched_internal = len(final_unmatched_internal)
total_unmatched_bank = len(final_unmatched_bank)

print(f"\nReconciliation Summary (Zenith Nigeria):")
print(f"Total Internal Credit Records: {len(zenith_ng_hex_df_recon)}")
print(f"Total Bank Credit Records: {len(zenith_ng_bank_df_recon)}")
print(f"Exact Matched Transactions: {len(matched_transactions)}")
print(f"Transactions Matched with Date Tolerance: {len(matched_with_tolerance_df)}")
print(f"Total Matched Transactions: {total_matched}")
print(f"Remaining Unmatched Internal Records: {total_unmatched_internal}")
print(f"Remaining Unmatched Bank Records: {total_unmatched_bank}")

if not matched_with_tolerance_df.empty:
    print("\nTransactions Matched with Date Tolerance:")
    print(matched_with_tolerance_df[['Date_Match_Internal', 'Amount_Internal', 
                                   'Date_Match_Bank', 'Amount_Bank']])

if not final_unmatched_internal.empty:
    print("\nUnmatched Internal Records:")
    print(final_unmatched_internal[['Date_Match', 'Amount_Internal']].head())

if not final_unmatched_bank.empty:
    print("\nUnmatched Bank Records:")
    print(final_unmatched_bank[['Date_Match', 'Amount_Bank']].head())

Total Internal Credit Amount (Zenith NG): 932,400,000.00
Total Bank Statement Credit Amount (Zenith NG): 932,800,000.00
Overall Discrepancy (Internal Credits - Bank Credits): -400,000.00

Reconciliation Summary (Zenith Nigeria):
Total Internal Credit Records: 5
Total Bank Credit Records: 5
Exact Matched Transactions: 3
Transactions Matched with Date Tolerance: 0
Total Matched Transactions: 3
Remaining Unmatched Internal Records: 2
Remaining Unmatched Bank Records: 2

Unmatched Internal Records:
   Date_Match  Amount_Internal
3  2025-06-13      154000000.0
6  2025-06-19      154700000.0

Unmatched Bank Records:
   Date_Match  Amount_Bank
4  2025-06-13  154500000.0
5  2025-06-19  154600000.0


## FW

In [ ]:
# 1. Load the datasets
FW_ng_hex_df = pd.read_csv('/Users/gracegitau/Downloads/#Recon May/NGN/FW Hex.csv')
FW_ng_bank_df = pd.read_csv('/Users/gracegitau/Downloads/#Recon May/NGN/FW Bank.csv', header=0)

# 2. Preprocessing for internal_df (Internal Records)
FW_ng_hex_df.columns = FW_ng_hex_df.columns.str.strip()

FW_ng_hex_df = FW_ng_hex_df.rename(columns={
    'TRANSFER_DATE': 'Date',
    'AMOUNT': 'Amount',
    'COMMENT': 'Description',
    'TRANSFER_ID': 'ID'
})

FW_ng_hex_df['Date'] = pd.to_datetime(FW_ng_hex_df['Date'])
FW_ng_hex_df_recon = FW_ng_hex_df[FW_ng_hex_df['Amount'] > 0].copy()
FW_ng_hex_df_recon = FW_ng_hex_df_recon[['Date', 'Amount', 'Description', 'ID']].copy()
FW_ng_hex_df_recon['Date_Match'] = FW_ng_hex_df_recon['Date'].dt.date

# 3. Preprocessing for bank_df (Bank Statements) - FLUTTERWAVE SPECIFIC
FW_ng_bank_df.columns = FW_ng_bank_df.columns.str.strip()

# Filter for Credits section (type='C') and exclude reversals
FW_ng_bank_df = FW_ng_bank_df[
    (FW_ng_bank_df['type'] == 'C') & 
    (~FW_ng_bank_df['remarks'].str.contains('rvsl', case=False, na=False))
].copy()

# Rename columns for consistency
FW_ng_bank_df = FW_ng_bank_df.rename(columns={
    'date': 'Date',
    'amount': 'Credit',
    'reference': 'Transaction_ID',
    'remarks': 'Description'
})

# Convert 'Date' to datetime and remove timezone
FW_ng_bank_df['Date'] = pd.to_datetime(FW_ng_bank_df['Date']).dt.tz_localize(None)

# Process Credit Amount - ensure numeric format
FW_ng_bank_df['Credit'] = (
    FW_ng_bank_df['Credit'].astype(str)
    .str.replace(',', '', regex=False)
    .replace('', '0')
    .astype(float)
)

# Filter for positive credits only
FW_ng_bank_df = FW_ng_bank_df[FW_ng_bank_df['Credit'] > 0].copy()

# For reconciliation, use the credit values
FW_ng_bank_df['Amount'] = FW_ng_bank_df['Credit']

# Select relevant columns for reconciliation
FW_ng_bank_df_recon = FW_ng_bank_df[['Date', 'Amount', 'Description', 'Transaction_ID']].copy()

# Extract only the date component for matching
FW_ng_bank_df_recon['Date_Match'] = FW_ng_bank_df_recon['Date'].dt.date

# 4. Calculate Total Amounts and Discrepancy
total_internal_credits = FW_ng_hex_df_recon['Amount'].sum()
total_bank_credits = FW_ng_bank_df_recon['Amount'].sum()
discrepancy_amount = total_internal_credits - total_bank_credits

print(f"Total Internal Credit Amount (Flutterwave NG): {total_internal_credits:,.2f}")
print(f"Total Bank Statement Credit Amount (Flutterwave NG): {total_bank_credits:,.2f}")
print(f"Overall Discrepancy: {discrepancy_amount:,.2f}")

# 5. Reconciliation (transaction-level) with Date Tolerance
# Ensure Date_Match columns have consistent types
FW_ng_hex_df_recon['Date_Match'] = FW_ng_hex_df_recon['Date_Match'].astype(str)
FW_ng_bank_df_recon['Date_Match'] = FW_ng_bank_df_recon['Date_Match'].astype(str)

FW_ng_hex_df_recon['Amount_Rounded'] = FW_ng_hex_df_recon['Amount'].round(2)
FW_ng_bank_df_recon['Amount_Rounded'] = FW_ng_bank_df_recon['Amount'].round(2)

reconciled_df = pd.merge(
    FW_ng_hex_df_recon.assign(Source_Internal='Internal'),
    FW_ng_bank_df_recon.assign(Source_Bank='Bank'),
    on=['Date_Match', 'Amount_Rounded'],
    how='outer',
    suffixes=('_Internal', '_Bank')
)

matched_transactions = reconciled_df.dropna(subset=['Source_Internal', 'Source_Bank'])

date_tolerance = pd.Timedelta(days=3)
unmatched_internal = reconciled_df[reconciled_df['Source_Bank'].isna()].copy()
unmatched_bank = reconciled_df[reconciled_df['Source_Internal'].isna()].copy()

unmatched_internal['Date_Match_DT'] = pd.to_datetime(unmatched_internal['Date_Match'])
unmatched_bank['Date_Match_DT'] = pd.to_datetime(unmatched_bank['Date_Match'])

matched_with_tolerance = []
matched_internal_indices = []
matched_bank_indices = []

for idx, internal_row in unmatched_internal.iterrows():
    internal_date = internal_row['Date_Match_DT']
    internal_amount = internal_row['Amount_Rounded']
    
    potential_matches = unmatched_bank[
        (unmatched_bank['Amount_Rounded'] == internal_amount) &
        (abs(unmatched_bank['Date_Match_DT'] - internal_date) <= date_tolerance)
    ]
    
    if not potential_matches.empty:
        bank_match = potential_matches.iloc[0]
        matched_with_tolerance.append({
            'Date_Internal': internal_row['Date_Internal'],
            'Amount_Internal': internal_row['Amount_Internal'],
            'Date_Match_Internal': internal_row['Date_Match'],
            'Date_Bank': bank_match['Date_Bank'],
            'Amount_Bank': bank_match['Amount_Bank'],
            'Date_Match_Bank': bank_match['Date_Match'],
            'Amount_Rounded': internal_amount
        })
        matched_internal_indices.append(idx)
        matched_bank_indices.append(bank_match.name)

matched_with_tolerance_df = pd.DataFrame(matched_with_tolerance)
final_unmatched_internal = unmatched_internal.drop(matched_internal_indices)
final_unmatched_bank = unmatched_bank.drop(matched_bank_indices)

# Merge same-day internal transactions before matching with bank
if len(final_unmatched_internal) > 0:
    # Group internal transactions by date and sum amounts
    internal_aggregated = (
        final_unmatched_internal.groupby('Date_Match')
        .agg({'Amount_Internal': 'sum'})
        .reset_index()
    )
    internal_aggregated['Amount_Rounded'] = internal_aggregated['Amount_Internal'].round(2)
    internal_aggregated['Date_Match_DT'] = pd.to_datetime(internal_aggregated['Date_Match'])
    
    # Try matching aggregated internal transactions with bank
    bank_unmatched = final_unmatched_bank.copy()
    bank_unmatched['Date_Match_DT'] = pd.to_datetime(bank_unmatched['Date_Match'])
    
    matched_aggregated = []
    matched_bank_indices = []
    
    for idx, internal_row in internal_aggregated.iterrows():
        internal_date = internal_row['Date_Match_DT']
        internal_amount = internal_row['Amount_Rounded']
        
        potential_matches = bank_unmatched[
            (bank_unmatched['Amount_Rounded'] == internal_amount) &
            (bank_unmatched['Date_Match_DT'] == internal_date)
        ]
        
        if not potential_matches.empty:
            bank_match = potential_matches.iloc[0]
            matched_aggregated.append({
                'Date_Match': internal_row['Date_Match'],
                'Aggregated_Internal_Amount': internal_row['Amount_Internal'],
                'Bank_Amount': bank_match['Amount_Bank'],
                'Bank_Date': bank_match['Date_Match']
            })
            matched_bank_indices.append(bank_match.name)
    
    # Update unmatched records
    if matched_aggregated:
        matched_aggregated_df = pd.DataFrame(matched_aggregated)
        final_unmatched_bank = final_unmatched_bank.drop(matched_bank_indices)
        
        # Remove the matched internal transactions
        matched_dates = matched_aggregated_df['Date_Match'].unique()
        final_unmatched_internal = final_unmatched_internal[
            ~final_unmatched_internal['Date_Match'].isin(matched_dates)
        ]
        
        print("\nAdditional Matches Found After Aggregating Same-Day Internal Transactions:")
        print(matched_aggregated_df)
        
        # Update matched counts
        total_matched += len(matched_aggregated_df)
        total_unmatched_internal = len(final_unmatched_internal)
        total_unmatched_bank = len(final_unmatched_bank)


# 6. Summary of Reconciliation
total_matched = len(matched_transactions) + len(matched_with_tolerance_df)
total_unmatched_internal = len(final_unmatched_internal)
total_unmatched_bank = len(final_unmatched_bank)

print(f"\nReconciliation Summary:")
print(f"Total Internal Credit Records: {len(FW_ng_hex_df_recon)}")
print(f"Total Bank Credit Records: {len(FW_ng_bank_df_recon)}")
print(f"Exact Matched Transactions: {len(matched_transactions)}")
print(f"Transactions Matched with Date Tolerance: {len(matched_with_tolerance_df)}")
print(f"Total Matched Transactions: {total_matched}")
print(f"Remaining Unmatched Internal Records: {total_unmatched_internal}")
print(f"Remaining Unmatched Bank Records: {total_unmatched_bank}")

if not matched_with_tolerance_df.empty:
    print("\nTransactions Matched with Date Tolerance:")
    print(matched_with_tolerance_df[['Date_Match_Internal', 'Amount_Internal', 
                                   'Date_Match_Bank', 'Amount_Bank']])

if not final_unmatched_internal.empty:
    print("\nUnmatched Internal Records:")
    print(final_unmatched_internal[['Date_Match', 'Amount_Internal']].head())

if not final_unmatched_bank.empty:
    print("\nUnmatched Bank Records:")
    print(final_unmatched_bank[['Date_Match', 'Amount_Bank']].head())

Total Internal Credit Amount (Flutterwave NG): 10,550,875,000.00
Total Bank Statement Credit Amount (Flutterwave NG): 10,550,875,000.00
Overall Discrepancy: 0.00

Additional Matches Found After Aggregating Same-Day Internal Transactions:
   Date_Match  Aggregated_Internal_Amount   Bank_Amount   Bank_Date
0  2025-06-04                9.528000e+08  9.528000e+08  2025-06-04
1  2025-06-05                2.143000e+09  2.143000e+09  2025-06-05
2  2025-06-13                9.408000e+08  9.408000e+08  2025-06-13

Reconciliation Summary (Flutterwave Nigeria):
Total Internal Credit Records: 20
Total Bank Credit Records: 17
Exact Matched Transactions: 14
Transactions Matched with Date Tolerance: 0
Total Matched Transactions: 14
Remaining Unmatched Internal Records: 0
Remaining Unmatched Bank Records: 0


## MP

In [159]:
import pandas as pd

# 1. Load the datasets - MoniePoint specific multi-sheet handling with header fix
def load_moniepoint_bank_statement(filepath):
    try:
        # Get all sheet names
        xls = pd.ExcelFile(filepath, engine='xlrd')
        sheet_names = xls.sheet_names
        
        if not sheet_names:
            print("No sheets found in the Excel file.")
            return pd.DataFrame()

        print(f"Loading {len(sheet_names)} sheets from {filepath}...")

        # Read the first sheet with header
        df_first_sheet = pd.read_excel(filepath, sheet_name=sheet_names[0], engine='xlrd', header=0)
        columns = df_first_sheet.columns.tolist() # Get columns from the first sheet
        print(f"Columns derived from first sheet ({sheet_names[0]}): {columns}")
        print(f"Shape of first sheet: {df_first_sheet.shape}")


        # Store all processed DataFrames
        all_dfs = [df_first_sheet]

        # Read the remaining sheets without header and assign columns
        for i in range(1, len(sheet_names)):
            sheet_name = sheet_names[i]
            # Read without header, then assign columns from the first sheet
            df_other_sheet = pd.read_excel(filepath, sheet_name=sheet_name, engine='xlrd', header=None)
            
            # Ensure the number of columns matches before assigning
            if df_other_sheet.shape[1] == len(columns):
                df_other_sheet.columns = columns
                all_dfs.append(df_other_sheet)
                print(f"Appended sheet '{sheet_name}' with shape {df_other_sheet.shape}")
            else:
                print(f"Warning: Sheet '{sheet_name}' has {df_other_sheet.shape[1]} columns, expected {len(columns)}. Skipping due to column mismatch.")
                # You might want to handle this differently, e.g., inspect the sheet
                # or try to force alignment if the mismatch is minor.
                # For now, we'll skip sheets that don't match the column count.

        # Combine all sheets into one DataFrame
        df_combined = pd.concat(all_dfs, ignore_index=True)
        
        print(f"Successfully combined {len(all_dfs)} sheets into a single DataFrame with shape: {df_combined.shape}")
        return df_combined
    except Exception as e:
        print(f"Error loading MoniePoint bank statement: {str(e)}")
        raise

# Load internal records and bank statements
MP_ng_hex_df = pd.read_csv('/Users/gracegitau/Downloads/#Recon May/NGN/MP Hex.csv')
MP_ng_bank_df = load_moniepoint_bank_statement('/Users/gracegitau/Downloads/#Recon May/NGN/MP Bank.xls')

# 2. Preprocessing for internal_df (Internal Records)
MP_ng_hex_df.columns = MP_ng_hex_df.columns.str.strip()

MP_ng_hex_df = MP_ng_hex_df.rename(columns={
    'TRANSFER_DATE': 'Date',
    'AMOUNT': 'Amount',
    'COMMENT': 'Description',
    'TRANSFER_ID': 'ID'
})

MP_ng_hex_df['Date'] = pd.to_datetime(MP_ng_hex_df['Date'])
MP_ng_hex_df_recon = MP_ng_hex_df[MP_ng_hex_df['Amount'] > 0].copy()
MP_ng_hex_df_recon = MP_ng_hex_df_recon[['Date', 'Amount', 'Description', 'ID']].copy()
MP_ng_hex_df_recon['Date_Match'] = MP_ng_hex_df_recon['Date'].dt.date

# Check for duplicates in internal records
duplicate_internal_records = MP_ng_hex_df_recon[MP_ng_hex_df_recon.duplicated(subset=['Description'], keep=False)]
if not duplicate_internal_records.empty:
    print("\nDuplicate records found in Internal Records:")
    for index, row in duplicate_internal_records.iterrows():
        print(f"ID: {row['ID']}, Amount: {row['Amount']}, Date: {row['Date']}, Comment: {row['Description']}")
else:
    print("\nNo duplicate records found in Internal Records.")

# 3. Preprocessing for bank_df (Bank Statements) - MONIEPOINT SPECIFIC
# This function will now receive the *already correctly combined* DataFrame
def preprocess_moniepoint_bank(df):
    print(f"\nStarting bank statement preprocessing with combined shape: {df.shape}")
    print(f"Columns BEFORE stripping: {df.columns.tolist()}") # DEBUG
    # Clean column names
    df.columns = df.columns.str.strip()
    print(f"Columns AFTER stripping: {df.columns.tolist()}") # DEBUG
    print(f"Shape AFTER stripping: {df.shape}") # DEBUG

    # Ensure 'DATE' column exists and is datetime
    if 'DATE' not in df.columns:
        print("Error: 'DATE' column not found in the bank statement. Cannot proceed with date filtering.")
        return pd.DataFrame()
    
    df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce') # 'coerce' turns unparseable dates into NaT
    df = df.dropna(subset=['DATE']).copy() # Drop rows where DATE conversion failed
    
    # Filter for current month (June)
    df = df[df['DATE'].dt.month == 6].copy()
    print(f"Shape after date filtering (June): {df.shape}")
    
    # Apply filters
    # Ensure all required columns exist before filtering
    required_cols_for_filters = ['TRANSACTION_TYPE', 'NARRATION', 'REFERENCE', 'AMOUNT'] # Added 'AMOUNT' here for safety
    if not all(col in df.columns for col in required_cols_for_filters):
        missing = [col for col in required_cols_for_filters if col not in df.columns]
        print(f"Error: Missing required columns for filtering: {missing}. Returning empty DataFrame.")
        return pd.DataFrame()

    df_cleaned = df[
        (df['TRANSACTION_TYPE'] == 'CREDIT') & 
        (df['NARRATION'].str.contains('MFY-WT', na=False)) & 
        (~df['REFERENCE'].str.contains('RVSL', na=False))
    ].copy()
    print(f"Shape after TRANSACTION_TYPE, NARRATION, REFERENCE filters: {df_cleaned.shape}")

    if df_cleaned.empty:
        print("No records found after applying all bank statement filters. Returning empty DataFrame.")
        return pd.DataFrame()
    
    # Extract sender name from narration
    def extract_sender_name(narration):
        narration = str(narration).lower()
        if 'verto financial tech' in narration or 'paga' in narration:
            return 'VertoFX, NGN a/c'
        elif 'sendfirst' in narration or 'duplo ltd' in narration:
            return 'Duplo, NGN a/c'
        elif 'esca' in narration:
            return 'Esca Nigeria, NGN a/c'
        elif 'resrv' in narration:
            return 'Resrv FX, NGN a/c'
        elif 'waza' in narration:
            return 'Waza, Nigeria, NGN a/c'
        elif 'flutterwave' in narration:
            return 'Flutterwave, NGN a/c'
        elif 'inexass' in narration:
            return 'AZA Finance, NGN a/c'
        elif 'nala' in narration:
            return 'Nala Payments'
        elif 'south one' in narration:
            return 'Southone NGN a/c'
        elif 'titan-paystack' in narration or 'multigate' in narration:
            return 'Multigate, NGN a/c'
        elif 'zerozilo' in narration or 'silverfile' in narration or 'palm bills' in narration:
            return 'Fincra, NGN a/c'
        elif 'ift technologies' in narration or 'budpay' in narration or 'bud infrastructure' in narration:
            return 'Torus Mara, NGN a/c'
        elif 'starks associates limited' in narration or 'shamiri' in narration or 'second jeu' in narration:
            return 'Straitpay (Starks), UK, NGN a/c'
        else:
            return 'Unknown'
    
    df_cleaned['SENDER_NAME'] = df_cleaned['NARRATION'].apply(extract_sender_name)
    
    # Combine split transactions within 30-minute windows
    # Ensure 'AMOUNT' column exists before grouping
    if 'AMOUNT' not in df_cleaned.columns:
        print("Error: 'AMOUNT' column not found for aggregation. Returning empty DataFrame.")
        return pd.DataFrame()

    df_combined = df_cleaned.groupby(['SENDER_NAME', pd.Grouper(key='DATE', freq='30min')]).agg({
        'AMOUNT': 'sum',
        'NARRATION': lambda x: ' '.join(x.dropna().unique()),
        'REFERENCE': lambda x: ' '.join(x.dropna().unique()),
        'TRANSACTION_TYPE': 'first',
    }).reset_index()
    
    df_combined.rename(columns={'DATE': 'DATE_TIME_WINDOW_START', 'AMOUNT': 'Amount'}, inplace=True)
    print(f"Shape after grouping by SENDER_NAME and 30min window: {df_combined.shape}")
    return df_combined

# Only proceed with preprocessing if the bank statement was loaded successfully
if not MP_ng_bank_df.empty:
    MP_ng_bank_df_recon = preprocess_moniepoint_bank(MP_ng_bank_df)
    if not MP_ng_bank_df_recon.empty:
        MP_ng_bank_df_recon['Date_Match'] = MP_ng_bank_df_recon['DATE_TIME_WINDOW_START'].dt.date
    else:
        print("MP_ng_bank_df_recon is empty after preprocessing.")
else:
    MP_ng_bank_df_recon = pd.DataFrame()
    print("MP_ng_bank_df is empty, skipping bank statement preprocessing.")


# 4. Calculate Total Amounts and Discrepancy
total_internal_credits = MP_ng_hex_df_recon['Amount'].sum()
total_bank_credits = MP_ng_bank_df_recon['Amount'].sum() if not MP_ng_bank_df_recon.empty else 0
discrepancy_amount = total_internal_credits - total_bank_credits

print(f"\nTotal Internal Credit Amount: {total_internal_credits:,.2f}")
print(f"Total Bank Statement Credit Amount: {total_bank_credits:,.2f}")
print(f"Overall Discrepancy : {discrepancy_amount:,.2f}")

# 5. Reconciliation (transaction-level) with Date Tolerance
MP_ng_hex_df_recon['Amount_Rounded'] = MP_ng_hex_df_recon['Amount'].round(2)

if not MP_ng_bank_df_recon.empty:
    MP_ng_bank_df_recon['Amount_Rounded'] = MP_ng_bank_df_recon['Amount'].round(2)

    # First try matching by amount and exact date
    reconciled_df = pd.merge(
        MP_ng_hex_df_recon.assign(Source_Internal='Internal'),
        MP_ng_bank_df_recon.assign(Source_Bank='Bank'),
        on=['Date_Match', 'Amount_Rounded'],
        how='outer',
        suffixes=('_Internal', '_Bank')
    )
    
    # Debug prints
    print("\nColumns in reconciled_df:", reconciled_df.columns.tolist())

    matched_transactions = reconciled_df.dropna(subset=['Source_Internal', 'Source_Bank'])

    date_tolerance = pd.Timedelta(days=3)
    unmatched_internal = reconciled_df[reconciled_df['Source_Bank'].isna()].copy()
    unmatched_bank = reconciled_df[reconciled_df['Source_Internal'].isna()].copy()

    # Debug prints
    print("\nColumns in unmatched_internal:", unmatched_internal.columns.tolist())
    print("Columns in unmatched_bank:", unmatched_bank.columns.tolist())


    unmatched_internal['Date_Match_DT'] = pd.to_datetime(unmatched_internal['Date_Match'])
    unmatched_bank['Date_Match_DT'] = pd.to_datetime(unmatched_bank['Date_Match'])

    matched_with_tolerance = []
    matched_internal_indices = []
    matched_bank_indices = []

    for idx, internal_row in unmatched_internal.iterrows():
        internal_date = internal_row['Date_Match_DT']
        internal_amount = internal_row['Amount_Rounded']
        
        potential_matches = unmatched_bank[
            (unmatched_bank['Amount_Rounded'] == internal_amount) &
            (abs(unmatched_bank['Date_Match_DT'] - internal_date) <= date_tolerance)
        ]
        
        if not potential_matches.empty:
            bank_match = potential_matches.iloc[0]
            matched_with_tolerance.append({
                # Corrected column names as discussed in the previous turn
                'Date_Internal': internal_row['Date'], 
                'Amount_Internal': internal_row['Amount_Internal'], 
                'Description_Internal': internal_row['Description'], 
                'ID_Internal': internal_row['ID'],
                
                'Date_Match_Internal': internal_row['Date_Match'], 

                'Date_Bank': bank_match['DATE_TIME_WINDOW_START'], 
                'Amount_Bank': bank_match['Amount_Bank'], 
                
                'Date_Match_Bank': bank_match['Date_Match'], 
                'Amount_Rounded': internal_amount
            })
            matched_internal_indices.append(idx)
            matched_bank_indices.append(bank_match.name)

    matched_with_tolerance_df = pd.DataFrame(matched_with_tolerance)
    final_unmatched_internal = unmatched_internal.drop(matched_internal_indices, errors='ignore')
    final_unmatched_bank = unmatched_bank.drop(matched_bank_indices, errors='ignore')

    # 6. Summary of Reconciliation
    total_matched = len(matched_transactions) + len(matched_with_tolerance_df)
    total_unmatched_internal = len(final_unmatched_internal)
    total_unmatched_bank = len(final_unmatched_bank)

    print(f"\nReconciliation Summary:")
    print(f"Total Internal Credit Records: {len(MP_ng_hex_df_recon)}")
    print(f"Total Bank Credit Records: {len(MP_ng_bank_df_recon)}")
    print(f"Exact Matched Transactions: {len(matched_transactions)}")
    print(f"Transactions Matched with Date Tolerance: {len(matched_with_tolerance_df)}")
    print(f"Total Matched Transactions: {total_matched}")
    print(f"Remaining Unmatched Internal Records: {total_unmatched_internal}")
    print(f"Remaining Unmatched Bank Records: {total_unmatched_bank}")

    if not matched_with_tolerance_df.empty:
        print("\nTransactions Matched with Date Tolerance:")
        print(matched_with_tolerance_df[['Date_Match_Internal', 'Amount_Internal', 
                                       'Date_Match_Bank', 'Amount_Bank']])

    if not final_unmatched_internal.empty:
        print("\nUnmatched Internal Records:")
        print(final_unmatched_internal[['Date_Match', 'Amount_Internal']].head())

    if not final_unmatched_bank.empty:
        print("\nUnmatched Bank Records:")
        print(final_unmatched_bank[['Date_Match', 'Amount_Bank']].head())
else:
    print("\nSkipping reconciliation steps as no bank statement data was loaded or processed successfully.")


# After your existing matching logic, add this:
if not final_unmatched_internal.empty and not final_unmatched_bank.empty:
    print("\nAnalyzing unmatched transactions with clustering...")
    
    # Ensure we have the SENDER_NAME column in bank unmatched
    final_unmatched_bank['SENDER_NAME'] = final_unmatched_bank['NARRATION'].apply(extract_sender_name)
    
    # Run clustering analysis
    clusters_df, discrepancies_df = cluster_unmatched_transactions(
        final_unmatched_internal,
        final_unmatched_bank
    )
    
    # Print summary
    if not clusters_df.empty:
        print("\nTransaction Clusters Summary:")
        print(clusters_df)
        
        if not discrepancies_df.empty:
            print("\nDiscrepancies Found:")
            print(discrepancies_df)
            
            # Calculate total discrepancies by type
            under_payment = discrepancies_df[discrepancies_df['type'] == 'under']['difference'].sum()
            over_payment = discrepancies_df[discrepancies_df['type'] == 'over']['difference'].sum()
            
            print(f"\nTotal Under-payment by Bank: {under_payment:,.2f}")
            print(f"Total Over-payment by Bank: {abs(over_payment):,.2f}")
            
            # Compare with original discrepancy
            calculated_discrepancy = under_payment + over_payment
            print(f"\nCalculated Discrepancy from Clusters: {calculated_discrepancy:,.2f}")
            print(f"Original Discrepancy: {discrepancy_amount:,.2f}")
            print(f"Unaccounted Difference: {discrepancy_amount - calculated_discrepancy:,.2f}")
    else:
        print("No clusters found in unmatched transactions.")

Loading 3 sheets from /Users/gracegitau/Downloads/#Recon May/NGN/MP Bank.xls...
Columns derived from first sheet (Monnify_Account_Statement): ['AMOUNT', 'BALANCE_BEFORE', 'BALANCE_AFTER', 'REFERENCE', 'DATE', 'TRANSACTION_TYPE', 'NARRATION']
Shape of first sheet: (65535, 7)
Appended sheet 'Monnify_Account_Statement 2' with shape (65536, 7)
Appended sheet 'Monnify_Account_Statement 3' with shape (33805, 7)
Successfully combined 3 sheets into a single DataFrame with shape: (164876, 7)

Duplicate records found in Internal Records:
ID: fx-transfer-2z36clGZ7MtbiYoLQTM77cK633P, Amount: 429800000.0, Date: 2025-06-26 00:00:00, Comment: fx-deal-quote-2z2jye54GJcgyUcTAq4cKtJIWkz
ID: fx-transfer-2z39wSNPNxBrmMu8GYZiYeF5Bkd, Amount: 625200000.0, Date: 2025-06-26 00:00:00, Comment: fx-deal-quote-2z2jye54GJcgyUcTAq4cKtJIWkz

Starting bank statement preprocessing with combined shape: (164876, 7)
Columns BEFORE stripping: ['AMOUNT', 'BALANCE_BEFORE', 'BALANCE_AFTER', 'REFERENCE', 'DATE', 'TRANSACTION_

KeyError: 'Amount'

In [ ]:
# 1. Data Loading Functions
def load_moniepoint_bank_statement(filepath):
    """Load and combine multi-sheet MoniePoint bank statement"""
    try:
        xls = pd.ExcelFile(filepath, engine='xlrd')
        sheet_names = xls.sheet_names
        
        if not sheet_names:
            print("No sheets found in the Excel file.")
            return pd.DataFrame()

        print(f"Loading {len(sheet_names)} sheets from {filepath}...")

        # Read first sheet to get columns
        df_first_sheet = pd.read_excel(filepath, sheet_name=sheet_names[0], engine='xlrd', header=0)
        columns = df_first_sheet.columns.tolist()
        print(f"Columns from first sheet ({sheet_names[0]}): {columns}")

        # Process all sheets
        all_dfs = [df_first_sheet]
        for sheet_name in sheet_names[1:]:
            df_sheet = pd.read_excel(filepath, sheet_name=sheet_name, engine='xlrd', header=None)
            if df_sheet.shape[1] == len(columns):
                df_sheet.columns = columns
                all_dfs.append(df_sheet)
                print(f"Appended {sheet_name} with shape {df_sheet.shape}")
            else:
                print(f"Warning: Skipping {sheet_name} - column mismatch")

        df_combined = pd.concat(all_dfs, ignore_index=True)
        print(f"Combined shape: {df_combined.shape}")
        return df_combined
    except Exception as e:
        print(f"Error loading bank statement: {str(e)}")
        raise

# 2. Sender Identification Functions
def extract_sender_name(narration):
    """Extract sender name from bank narration"""
    narration = str(narration).lower()
    if 'verto financial tech' in narration or 'paga' in narration:
        return 'VertoFX, NGN a/c'
    elif 'sendfirst' in narration or 'duplo ltd' in narration:
        return 'Duplo, NGN a/c'
    elif 'esca' in narration:
        return 'Esca Nigeria, NGN a/c'
    elif 'resrv' in narration:
        return 'Resrv FX, NGN a/c'
    elif 'waza' in narration:
        return 'Waza, Nigeria, NGN a/c'
    elif 'flutterwave' in narration:
        return 'Flutterwave, NGN a/c'
    elif 'inexass' in narration:
        return 'AZA Finance, NGN a/c'
    elif 'nala' in narration:
        return 'Nala Payments'
    elif 'south one' in narration:
        return 'Southone NGN a/c'
    elif 'titan-paystack' in narration or 'multigate' in narration:
        return 'Multigate, NGN a/c'
    elif 'zerozilo' in narration or 'silverfile' in narration or 'palm bills' in narration:
        return 'Fincra, NGN a/c'
    elif 'ift technologies' in narration or 'budpay' in narration or 'bud infrastructure' in narration:
        return 'Torus Mara, NGN a/c'
    elif 'starks associates limited' in narration or 'shamiri' in narration or 'second jeu' in narration:
        return 'Straitpay (Starks), UK, NGN a/c'
    else:
        return 'Unknown'

def extract_internal_sender(description):
    """Extract sender name from internal description"""
    description = str(description).lower()
    if 'fx-deal' in description:
        for sender in ['verto', 'duplo', 'esca', 'resrv', 'waza', 'flutterwave']:
            if sender in description:
                return f"{sender.title()}, NGN a/c"
    elif 'payout' in description:
        return 'Internal Payout'
    return extract_sender_name(description)  # Fallback to bank patterns

# 3. Bank Statement Preprocessing
def preprocess_moniepoint_bank(df):
    """Clean and prepare bank statement data"""
    print(f"\nStarting bank preprocessing. Initial shape: {df.shape}")
    
    # Clean columns
    df.columns = df.columns.str.strip()
    
    # Convert and filter dates
    df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce')
    df = df.dropna(subset=['DATE']).copy()
    df = df[df['DATE'].dt.month == 6].copy()  # Filter for June
    
    # Apply business rules
    df_cleaned = df[
        (df['TRANSACTION_TYPE'] == 'CREDIT') & 
        (df['NARRATION'].str.contains('MFY-WT', na=False)) & 
        (~df['REFERENCE'].str.contains('RVSL', na=False))
    ].copy()
    
    if df_cleaned.empty:
        print("No records after filters")
        return pd.DataFrame()
    
    # Extract sender and combine transactions
    df_cleaned['SENDER_NAME'] = df_cleaned['NARRATION'].apply(extract_sender_name)
    
    df_combined = df_cleaned.groupby(['SENDER_NAME', pd.Grouper(key='DATE', freq='30min')]).agg({
        'AMOUNT': 'sum',
        'NARRATION': lambda x: ' '.join(x.dropna().unique()),
        'REFERENCE': lambda x: ' '.join(x.dropna().unique()),
    }).reset_index()
    
    df_combined.rename(columns={'DATE': 'DATE_TIME_WINDOW_START', 'AMOUNT': 'Amount'}, inplace=True)
    print(f"Final bank shape: {df_combined.shape}")
    return df_combined

In [158]:
def cluster_unmatched_transactions(unmatched_internal, unmatched_bank, date_tol=3):
    """Group unmatched transactions by sender and date proximity"""
    # Ensure date columns are datetime
    unmatched_internal['Date_Match'] = pd.to_datetime(unmatched_internal['Date_Match'])
    unmatched_bank['Date_Match'] = pd.to_datetime(unmatched_bank['Date_Match'])
    
    # Add sender names
    unmatched_internal['SENDER_NAME'] = unmatched_internal['Description'].apply(extract_internal_sender)
    unmatched_bank['SENDER_NAME'] = unmatched_bank['NARRATION'].apply(extract_sender_name)
    
    clusters = []
    discrepancies = []
    
    # Process by sender
    for sender in set(unmatched_internal['SENDER_NAME'].unique()).union(set(unmatched_bank['SENDER_NAME'].unique())):
        internal = unmatched_internal[unmatched_internal['SENDER_NAME'] == sender]
        bank = unmatched_bank[unmatched_bank['SENDER_NAME'] == sender]
        
        if internal.empty and bank.empty:
            continue
            
        # Find date clusters
        date_clusters = []
        used_bank_indices = set()
        
        for _, int_row in internal.iterrows():
            int_date = int_row['Date_Match']
            int_amount = int_row['Amount']
            
            # Find bank transactions within date tolerance
            bank_matches = bank[
                (abs((bank['Date_Match'] - int_date).dt.days <= date_tol) &
                (~bank.index.isin(used_bank_indices)))
            ]
            
            if not bank_matches.empty:
                # Create cluster
                cluster = {
                    'sender': sender,
                    'internal_ids': [int_row['ID']],
                    'bank_ids': bank_matches.index.tolist(),
                    'internal_amounts': [int_amount],
                    'bank_amounts': bank_matches['Amount'].tolist(),
                    'start_date': min(int_date, bank_matches['Date_Match'].min()),
                    'end_date': max(int_date, bank_matches['Date_Match'].max())
                }
                
                # Mark bank transactions as used
                used_bank_indices.update(bank_matches.index)
                date_clusters.append(cluster)
        
        # Analyze clusters
        for cluster in date_clusters:
            total_internal = sum(cluster['internal_amounts'])
            total_bank = sum(cluster['bank_amounts'])
            discrepancy = total_internal - total_bank
            
            clusters.append({
                'Sender': sender,
                'Date Range': f"{cluster['start_date'].date()} to {cluster['end_date'].date()}",
                'Internal Transactions': len(cluster['internal_amounts']),
                'Bank Transactions': len(cluster['bank_amounts']),
                'Total Internal': total_internal,
                'Total Bank': total_bank,
                'Discrepancy': discrepancy
            })
            
            if abs(discrepancy) > 1:  # Ignore tiny rounding differences
                discrepancies.append({
                    'Sender': sender,
                    'Period': f"{cluster['start_date'].date()} to {cluster['end_date'].date()}",
                    'Internal Total': total_internal,
                    'Bank Total': total_bank,
                    'Difference': discrepancy,
                    'Type': 'Under' if discrepancy > 0 else 'Over'
                })
    
    return pd.DataFrame(clusters), pd.DataFrame(discrepancies)

In [ ]:
# In the main reconciliation section:
if not MP_ng_bank_df_recon.empty:
    print("\nPerforming initial reconciliation...")
    
    # Ensure Date_Match is datetime in both DataFrames
    MP_ng_hex_df_recon['Date_Match'] = pd.to_datetime(MP_ng_hex_df_recon['Date_Match'])
    MP_ng_bank_df_recon['Date_Match'] = pd.to_datetime(MP_ng_bank_df_recon['Date_Match'])
    
    # Round amounts for matching
    MP_ng_hex_df_recon['Amount_Rounded'] = MP_ng_hex_df_recon['Amount'].round(2)
    MP_ng_bank_df_recon['Amount_Rounded'] = MP_ng_bank_df_recon['Amount'].round(2)
    
    # Rest of your reconciliation code...
    

In [163]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta

# 1. Data Loading Functions
def load_moniepoint_bank_statement(filepath):
    """Load and combine multi-sheet MoniePoint bank statement"""
    try:
        xls = pd.ExcelFile(filepath, engine='xlrd')
        sheet_names = xls.sheet_names
        
        if not sheet_names:
            print("No sheets found in the Excel file.")
            return pd.DataFrame()

        print(f"Loading {len(sheet_names)} sheets from {filepath}...")

        # Read first sheet to get columns
        df_first_sheet = pd.read_excel(filepath, sheet_name=sheet_names[0], engine='xlrd', header=0)
        columns = df_first_sheet.columns.tolist()
        print(f"Columns from first sheet ({sheet_names[0]}): {columns}")

        # Process all sheets
        all_dfs = [df_first_sheet]
        for sheet_name in sheet_names[1:]:
            df_sheet = pd.read_excel(filepath, sheet_name=sheet_name, engine='xlrd', header=None)
            if df_sheet.shape[1] == len(columns):
                df_sheet.columns = columns
                all_dfs.append(df_sheet)
                print(f"Appended {sheet_name} with shape {df_sheet.shape}")
            else:
                print(f"Warning: Skipping {sheet_name} - column mismatch")

        df_combined = pd.concat(all_dfs, ignore_index=True)
        print(f"Combined shape: {df_combined.shape}")
        return df_combined
    except Exception as e:
        print(f"Error loading bank statement: {str(e)}")
        raise

# 2. Sender Identification Functions
def extract_sender_name(narration):
    """Extract sender name from bank narration"""
    narration = str(narration).lower()
    if 'verto financial tech' in narration or 'paga' in narration:
        return 'VertoFX, NGN a/c'
    elif 'sendfirst' in narration or 'duplo ltd' in narration:
        return 'Duplo, NGN a/c'
    elif 'esca' in narration:
        return 'Esca Nigeria, NGN a/c'
    elif 'resrv' in narration:
        return 'Resrv FX, NGN a/c'
    elif 'waza' in narration:
        return 'Waza, Nigeria, NGN a/c'
    elif 'flutterwave' in narration:
        return 'Flutterwave, NGN a/c'
    elif 'inexass' in narration:
        return 'AZA Finance, NGN a/c'
    elif 'nala' in narration:
        return 'Nala Payments'
    elif 'south one' in narration:
        return 'Southone NGN a/c'
    elif 'titan-paystack' in narration or 'multigate' in narration:
        return 'Multigate, NGN a/c'
    elif 'zerozilo' in narration or 'silverfile' in narration or 'palm bills' in narration:
        return 'Fincra, NGN a/c'
    elif 'ift technologies' in narration or 'budpay' in narration or 'bud infrastructure' in narration:
        return 'Torus Mara, NGN a/c'
    elif 'starks associates limited' in narration or 'shamiri' in narration or 'second jeu' in narration:
        return 'Straitpay (Starks), UK, NGN a/c'
    else:
        return 'Unknown'

def extract_internal_sender(description):
    """Extract sender name from internal description"""
    description = str(description).lower()
    if 'fx-deal' in description:
        for sender in ['verto', 'duplo', 'esca', 'resrv', 'waza', 'flutterwave']:
            if sender in description:
                return f"{sender.title()}, NGN a/c"
    elif 'payout' in description:
        return 'Internal Payout'
    return extract_sender_name(description)  # Fallback to bank patterns

# 3. Bank Statement Preprocessing
def preprocess_moniepoint_bank(df):
    """Clean and prepare bank statement data"""
    print(f"\nStarting bank preprocessing. Initial shape: {df.shape}")
    
    # Clean columns
    df.columns = df.columns.str.strip()
    
    # Convert and filter dates
    df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce')
    df = df.dropna(subset=['DATE']).copy()
    df = df[df['DATE'].dt.month == 6].copy()  # Filter for June
    
    # Apply business rules
    df_cleaned = df[
        (df['TRANSACTION_TYPE'] == 'CREDIT') & 
        (df['NARRATION'].str.contains('MFY-WT', na=False)) & 
        (~df['REFERENCE'].str.contains('RVSL', na=False))
    ].copy()
    
    if df_cleaned.empty:
        print("No records after filters")
        return pd.DataFrame()
    
    # Extract sender and combine transactions
    df_cleaned['SENDER_NAME'] = df_cleaned['NARRATION'].apply(extract_sender_name)
    
    df_combined = df_cleaned.groupby(['SENDER_NAME', pd.Grouper(key='DATE', freq='30min')]).agg({
        'AMOUNT': 'sum',
        'NARRATION': lambda x: ' '.join(x.dropna().unique()),
        'REFERENCE': lambda x: ' '.join(x.dropna().unique()),
    }).reset_index()
    
    df_combined.rename(columns={'DATE': 'DATE_TIME_WINDOW_START', 'AMOUNT': 'Amount'}, inplace=True)
    df_combined['Date_Match'] = df_combined['DATE_TIME_WINDOW_START'].dt.date
    print(f"Final bank shape: {df_combined.shape}")
    return df_combined

# 4. Clustering for Unmatched Transactions
def cluster_unmatched_transactions(unmatched_internal, unmatched_bank, date_tol=3):
    """Group unmatched transactions by sender and date proximity"""
    # Ensure date columns are datetime
    unmatched_internal['Date_Match'] = pd.to_datetime(unmatched_internal['Date_Match'])
    unmatched_bank['Date_Match'] = pd.to_datetime(unmatched_bank['Date_Match'])
    
    # Add sender names
    unmatched_internal['SENDER_NAME'] = unmatched_internal['Description'].apply(extract_internal_sender)
    unmatched_bank['SENDER_NAME'] = unmatched_bank['NARRATION'].apply(extract_sender_name)
    
    clusters = []
    discrepancies = []
    
    # Process by sender
    for sender in set(unmatched_internal['SENDER_NAME'].unique()).union(set(unmatched_bank['SENDER_NAME'].unique())):
        internal = unmatched_internal[unmatched_internal['SENDER_NAME'] == sender]
        bank = unmatched_bank[unmatched_bank['SENDER_NAME'] == sender]
        
        if internal.empty and bank.empty:
            continue
            
        # Find date clusters
        date_clusters = []
        used_bank_indices = set()
        
        for _, int_row in internal.iterrows():
            int_date = int_row['Date_Match']
            int_amount = int_row['Amount']
            
            # Find bank transactions within date tolerance
            bank_matches = bank[
                (abs((bank['Date_Match'] - int_date).dt.days <= date_tol) &
                (~bank.index.isin(used_bank_indices)))
            ]
            
            if not bank_matches.empty:
                # Create cluster
                cluster = {
                    'sender': sender,
                    'internal_ids': [int_row['ID']],
                    'bank_ids': bank_matches.index.tolist(),
                    'internal_amounts': [int_amount],
                    'bank_amounts': bank_matches['Amount'].tolist(),
                    'start_date': min(int_date, bank_matches['Date_Match'].min()),
                    'end_date': max(int_date, bank_matches['Date_Match'].max())
                }
                
                # Mark bank transactions as used
                used_bank_indices.update(bank_matches.index)
                date_clusters.append(cluster)
        
        # Analyze clusters
        for cluster in date_clusters:
            total_internal = sum(cluster['internal_amounts'])
            total_bank = sum(cluster['bank_amounts'])
            discrepancy = total_internal - total_bank
            
            clusters.append({
                'Sender': sender,
                'Date Range': f"{cluster['start_date'].date()} to {cluster['end_date'].date()}",
                'Internal Transactions': len(cluster['internal_amounts']),
                'Bank Transactions': len(cluster['bank_amounts']),
                'Total Internal': total_internal,
                'Total Bank': total_bank,
                'Discrepancy': discrepancy
            })
            
            if abs(discrepancy) > 1:  # Ignore tiny rounding differences
                discrepancies.append({
                    'Sender': sender,
                    'Period': f"{cluster['start_date'].date()} to {cluster['end_date'].date()}",
                    'Internal Total': total_internal,
                    'Bank Total': total_bank,
                    'Difference': discrepancy,
                    'Type': 'Under' if discrepancy > 0 else 'Over'
                })
    
    return pd.DataFrame(clusters), pd.DataFrame(discrepancies)

# 5. Visualization Functions
def visualize_discrepancies(discrepancies_df):
    """Generate plots for discrepancy analysis"""
    if discrepancies_df.empty:
        print("No discrepancies to visualize")
        return
    
    plt.figure(figsize=(14, 6))
    
    # Under/over payment by sender
    plt.subplot(1, 2, 1)
    discrepancies_df.groupby(['Sender', 'Type'])['Difference'].sum().unstack().plot(
        kind='bar', stacked=True, color=['red', 'green']
    )
    plt.title("Discrepancies by Sender")
    plt.ylabel("Amount")
    plt.xticks(rotation=45, ha='right')
    
    # Time trend
    plt.subplot(1, 2, 2)
    discrepancies_df['Period'] = discrepancies_df['Period'].str.split(' to ').str[0]
    discrepancies_df['Period'] = pd.to_datetime(discrepancies_df['Period'])
    discrepancies_df.groupby(['Period', 'Type'])['Difference'].sum().unstack().plot(
        kind='line', marker='o', color=['red', 'green']
    )
    plt.title("Discrepancies Over Time")
    plt.ylabel("Amount")
    
    plt.tight_layout()
    plt.show()

# MAIN EXECUTION
if __name__ == "__main__":
    # 1. Load Data
    print("Loading data...")
    MP_ng_hex_df = pd.read_csv('/Users/gracegitau/Downloads/#Recon May/NGN/MP Hex.csv')
    MP_ng_bank_df = load_moniepoint_bank_statement('/Users/gracegitau/Downloads/#Recon May/NGN/MP Bank.xls')
    
    # 2. Preprocess Internal Records
    print("\nPreprocessing internal records...")
    MP_ng_hex_df.columns = MP_ng_hex_df.columns.str.strip()
    MP_ng_hex_df = MP_ng_hex_df.rename(columns={
        'TRANSFER_DATE': 'Date',
        'AMOUNT': 'Amount',
        'COMMENT': 'Description',
        'TRANSFER_ID': 'ID'
    })
    
    MP_ng_hex_df['Date'] = pd.to_datetime(MP_ng_hex_df['Date'])
    MP_ng_hex_df_recon = MP_ng_hex_df[MP_ng_hex_df['Amount'] > 0].copy()
    MP_ng_hex_df_recon = MP_ng_hex_df_recon[['Date', 'Amount', 'Description', 'ID']].copy()
    MP_ng_hex_df_recon['Date_Match'] = MP_ng_hex_df_recon['Date'].dt.date
    
    # Check duplicates
    duplicates = MP_ng_hex_df_recon[MP_ng_hex_df_recon.duplicated(subset=['Description'], keep=False)]
    if not duplicates.empty:
        print(f"\nFound {len(duplicates)} duplicate internal records:")
        print(duplicates[['ID', 'Amount', 'Date', 'Description']].head())
    
    # 3. Preprocess Bank Statements
    if not MP_ng_bank_df.empty:
        MP_ng_bank_df_recon = preprocess_moniepoint_bank(MP_ng_bank_df)
    else:
        MP_ng_bank_df_recon = pd.DataFrame()
        print("No bank data loaded")
    
    # 4. Initial Reconciliation
    if not MP_ng_bank_df_recon.empty:
        print("\nPerforming initial reconciliation...")
        # Round amounts for matching
        MP_ng_hex_df_recon['Amount_Rounded'] = MP_ng_hex_df_recon['Amount'].round(2)
        MP_ng_bank_df_recon['Amount_Rounded'] = MP_ng_bank_df_recon['Amount'].round(2)
        
        # Exact matches
        reconciled_df = pd.merge(
            MP_ng_hex_df_recon.assign(Source='Internal'),
            MP_ng_bank_df_recon.assign(Source='Bank'),
            on=['Date_Match', 'Amount_Rounded'],
            how='outer',
            suffixes=('_Internal', '_Bank')
        )
        
        matched = reconciled_df.dropna(subset=['Source_Internal', 'Source_Bank'])
        unmatched_internal = reconciled_df[reconciled_df['Source_Bank'].isna()].copy()
        unmatched_bank = reconciled_df[reconciled_df['Source_Internal'].isna()].copy()
        
        # Prepare columns for clustering
        unmatched_internal = unmatched_internal.rename(columns={
            'Date_Internal': 'Date',
            'Amount_Internal': 'Amount',
            'Description_Internal': 'Description',
            'ID_Internal': 'ID',
            'Date_Match_Internal': 'Date_Match'
        })[['Date', 'Amount', 'Description', 'ID', 'Date_Match']]
        
        unmatched_bank = unmatched_bank.rename(columns={
            'DATE_TIME_WINDOW_START_Bank': 'DATE_TIME_WINDOW_START',
            'Amount_Bank': 'Amount',
            'NARRATION_Bank': 'NARRATION',
            'SENDER_NAME_Bank': 'SENDER_NAME',
            'Date_Match_Bank': 'Date_Match'
        })[['DATE_TIME_WINDOW_START', 'Amount', 'NARRATION', 'SENDER_NAME', 'Date_Match']]
        
        # 5. Cluster Analysis for Unmatched
        if not unmatched_internal.empty and not unmatched_bank.empty:
            print("\nAnalyzing unmatched transactions with clustering...")
            clusters_df, discrepancies_df = cluster_unmatched_transactions(
                unmatched_internal,
                unmatched_bank
            )
            
            if not clusters_df.empty:
                print("\nTransaction Clusters Summary:")
                print(clusters_df.to_string())
                
                if not discrepancies_df.empty:
                    print("\nDiscrepancies Found:")
                    print(discrepancies_df.to_string())
                    
                    # Calculate totals
                    under = discrepancies_df[discrepancies_df['Type'] == 'Under']['Difference'].sum()
                    over = discrepancies_df[discrepancies_df['Type'] == 'Over']['Difference'].sum()
                    
                    print(f"\nSummary:")
                    print(f"Total Under-Payments: {under:,.2f}")
                    print(f"Total Over-Payments: {abs(over):,.2f}")
                    print(f"Net Discrepancy: {under + over:,.2f}")
                    
                    # Visualize
                    visualize_discrepancies(discrepancies_df)
            else:
                print("No clusters found in unmatched transactions")
        
        # 6. Final Reporting
        total_internal = MP_ng_hex_df_recon['Amount'].sum()
        total_bank = MP_ng_bank_df_recon['Amount'].sum()
        initial_discrepancy = total_internal - total_bank
        
        print("\nFinal Reconciliation Summary:")
        print(f"Total Internal Amount: {total_internal:,.2f}")
        print(f"Total Bank Amount: {total_bank:,.2f}")
        print(f"Initial Discrepancy: {initial_discrepancy:,.2f}")
        
        if 'discrepancies_df' in locals() and not discrepancies_df.empty:
            cluster_discrepancy = under + over
            print(f"Discrepancy from Clusters: {cluster_discrepancy:,.2f}")
            print(f"Unaccounted Difference: {initial_discrepancy - cluster_discrepancy:,.2f}")
    else:
        print("\nSkipping reconciliation - no valid bank data")

Loading data...
Loading 3 sheets from /Users/gracegitau/Downloads/#Recon May/NGN/MP Bank.xls...
Columns from first sheet (Monnify_Account_Statement): ['AMOUNT', 'BALANCE_BEFORE', 'BALANCE_AFTER', 'REFERENCE', 'DATE', 'TRANSACTION_TYPE', 'NARRATION']
Appended Monnify_Account_Statement 2 with shape (65536, 7)
Appended Monnify_Account_Statement 3 with shape (33805, 7)
Combined shape: (164876, 7)

Preprocessing internal records...

Found 2 duplicate internal records:
                                         ID       Amount       Date  \
7   fx-transfer-2z36clGZ7MtbiYoLQTM77cK633P  429800000.0 2025-06-26   
98  fx-transfer-2z39wSNPNxBrmMu8GYZiYeF5Bkd  625200000.0 2025-06-26   

                                  Description  
7   fx-deal-quote-2z2jye54GJcgyUcTAq4cKtJIWkz  
98  fx-deal-quote-2z2jye54GJcgyUcTAq4cKtJIWkz  

Starting bank preprocessing. Initial shape: (164876, 7)
Final bank shape: (140, 6)

Performing initial reconciliation...

Analyzing unmatched transactions with clustering...